#  Import Libraries

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import os
import ydata_profiling
from ydata_profiling import ProfileReport
from phik import phik_matrix
from phik import resources
from phik.report import plot_correlation_matrix
import plotly.graph_objects as go
from scipy.stats import chi2_contingency
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from itertools import combinations
import warnings
from sqlalchemy import create_engine, Column, String, Integer, Float, Date, ForeignKey, inspect
from sqlalchemy.orm import declarative_base, relationship

# Load the Data

In [ ]:
encounters = pd.read_csv('Data/encounters.csv')
patients = pd.read_csv('Data/patients.csv')
payers = pd.read_csv('Data/payers.csv')
procedures = pd.read_csv('Data/procedures.csv')
COVID_19_Test_to_Treat = pd.read_csv('Data/COVID-19_Test_to_Treat_20250108.csv')
Daily_Town_COVID19 = pd.read_csv('Data/Daily_Town_COVID19_MA-1.csv')

# Initial Exploration

## Display first few rows

In [ ]:
display(encounters.head())
display(patients.head())
display(payers.head())
display(procedures.head())
display(COVID_19_Test_to_Treat.head())
display(Daily_Town_COVID19.head())

,Id,START,STOP,PATIENT,ORGANIZATION,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,32c84703-2481-49cd-d571-3899d5820253,2011-01-02T09:26:36Z,2011-01-02T12:58:36Z,3de74169-7f67-9304-91d4-757e0f3a14d2,d78e84ec-30aa-3bba-a33a-f29a3a454662,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,185347001,Encounter for problem (procedure),85.55,1018.02,0.00,NaN,NaN
1,c98059da-320a-c0a6-fced-c8815f3e3f39,2011-01-03T05:44:39Z,2011-01-03T06:01:42Z,d9ec2e44-32e9-9148-179a-1653348cc4e2,d78e84ec-30aa-3bba-a33a-f29a3a454662,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,outpatient,308335008,Patient encounter procedure,142.58,2619.36,0.00,NaN,NaN
2,4ad28a3a-2479-782b-f29c-d5b3f41a001e,2011-01-03T14:32:11Z,2011-01-03T14:47:11Z,73babadf-5b2b-fee7-189e-6f41ff213e01,d78e84ec-30aa-3bba-a33a-f29a3a454662,7caa7254-5050-3b5e-9eae-bd5ea30e809c,outpatient,185349003,Encounter for check up (procedure),85.55,461.59,305.27,NaN,NaN
3,c3f4da61-e4b4-21d5-587a-fbc89943bc19,2011-01-03T16:24:45Z,2011-01-03T16:39:45Z,3b46a0b7-0f34-9b9a-c319-ace4a1f58c0b,d78e84ec-30aa-3bba-a33a-f29a3a454662,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,wellness,162673000,General examination of patient (procedure),136.80,1784.24,0.00,NaN,NaN
4,a9183b4f-2572-72ea-54c2-b3cd038b4be7,2011-01-03T17:36:53Z,2011-01-03T17:51:53Z,fa006887-d93c-d302-8b89-f3c25f88c0e1,d78e84ec-30aa-3bba-a33a-f29a3a454662,42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,ambulatory,390906007,Follow-up encounter,85.55,234.72,0.00,55822004.0,Hyperlipidemia


,Id,BIRTHDATE,DEATHDATE,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON
0,5605b66b-e92d-c16c-1b83-b8bf7040d51f,1977-03-19,NaN,Mrs.,Nikita578,Erdman779,NaN,Leannon79,M,white,nonhispanic,F,Wakefield Massachusetts US,510 Little Station Unit 69,Quincy,Massachusetts,Norfolk County,2186.0,42.290937,-70.975503
1,6e5ae27c-8038-7988-e2c0-25a103f01bfa,1940-02-19,NaN,Mr.,Zane918,Hodkiewicz467,NaN,NaN,M,white,nonhispanic,M,Brookline Massachusetts US,747 Conn Throughway,Boston,Massachusetts,Suffolk County,2135.0,42.308831,-71.063162
2,8123d076-0886-9007-e956-d5864aa121a7,1958-06-04,NaN,Mr.,Quinn173,Marquardt819,NaN,NaN,M,white,nonhispanic,M,Gardner Massachusetts US,816 Okuneva Extension Apt 91,Quincy,Massachusetts,Norfolk County,2170.0,42.265177,-70.967085
3,770518e4-6133-648e-60c9-071eb2f0e2ce,1928-12-25,2017-09-29,Mr.,Abel832,Smitham825,NaN,NaN,M,white,hispanic,M,Randolph Massachusetts US,127 Cole Way Unit 95,Boston,Massachusetts,Suffolk County,2118.0,42.334304,-71.066801
4,f96addf5-81b9-0aab-7855-d208d3d352c5,1928-12-25,2014-02-23,Mr.,Edwin773,Labadie908,NaN,NaN,M,white,hispanic,M,Stow Massachusetts US,976 Ziemann Gateway,Boston,Massachusetts,Suffolk County,2125.0,42.346771,-71.058813


,Id,NAME,ADDRESS,CITY,STATE_HEADQUARTERED,ZIP,PHONE
0,b3221cfc-24fb-339e-823d-bc4136cbc4ed,Dual Eligible,7500 Security Blvd,Baltimore,MD,21244.0,1-877-267-2323
1,7caa7254-5050-3b5e-9eae-bd5ea30e809c,Medicare,7500 Security Blvd,Baltimore,MD,21244.0,1-800-633-4227
2,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,Medicaid,7500 Security Blvd,Baltimore,MD,21244.0,1-877-267-2323
3,d47b3510-2895-3b70-9897-342d681c769d,Humana,500 West Main St,Louisville,KY,40018.0,1-844-330-7799
4,6e2f1a2d-27bd-3701-8d08-dae202c58632,Blue Cross Blue Shield,Michigan Plaza,Chicago,IL,60007.0,1-800-262-2583


,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION
0,2011-01-02T09:26:36Z,2011-01-02T12:58:36Z,3de74169-7f67-9304-91d4-757e0f3a14d2,32c84703-2481-49cd-d571-3899d5820253,265764009,Renal dialysis (procedure),903,NaN,NaN
1,2011-01-03T05:44:39Z,2011-01-03T06:01:42Z,d9ec2e44-32e9-9148-179a-1653348cc4e2,c98059da-320a-c0a6-fced-c8815f3e3f39,76601001,Intramuscular injection,2477,NaN,NaN
2,2011-01-04T14:49:55Z,2011-01-04T15:04:55Z,d856d6e6-4c98-e7a2-129b-44076c63d008,2cfd4ddd-ad13-fe1e-528b-15051cea2ec3,703423002,Combined chemotherapy and radiation therapy (p...,11620,363406005.0,Malignant tumor of colon
3,2011-01-05T04:02:09Z,2011-01-05T04:17:09Z,bc9d59c3-0a30-6e3b-f47d-022e4f03c8de,17966936-0878-f4db-128b-a43ae10d0878,173160006,Diagnostic fiberoptic bronchoscopy (procedure),9796,162573006.0,Suspected lung cancer (situation)
4,2011-01-05T12:58:36Z,2011-01-05T16:42:36Z,3de74169-7f67-9304-91d4-757e0f3a14d2,9de5f0b0-4ba4-ce6f-45fb-b55c202f31a5,265764009,Renal dialysis (procedure),1255,NaN,NaN


,Provider Name,Address1,Address2,City,State,Zip,Last Report Date,Public Website,Phone Number,Geopoint
0,PHARMAG PHARMACY,6622 SW 24 ST,NaN,MIAMI,FL,33155,01/30/2024 12:00:00 AM,NaN,NaN,POINT (-80.30304 25.74779)
1,SNEADS PHARMACY,7995 HWY 90,NaN,SNEADS,FL,32460,01/30/2024 12:00:00 AM,NaN,NaN,POINT (-84.928003 30.709665)
2,FEDCO DRUGS,255 SUNRISE AVE,NaN,PALM BEACH,FL,33480,01/30/2024 12:00:00 AM,NaN,NaN,POINT (-80.040021 26.720425)
3,LONGS DRUGS #09955,848 ALA LILIKOI ST,NaN,HONOLULU,HI,96818,01/30/2024 12:00:00 AM,NaN,NaN,POINT (-157.91342 21.345818)
4,ELKMONT PHARMACY,25462 HL HWY 127,NaN,ELKMONT,AL,35620,01/30/2024 12:00:00 AM,NaN,NaN,POINT (-86.988302 34.919333)


,Date,County,Town,covid_cases_cumulative,daily_cases
0,4/1/2020,Barnstable,Barnstable,47,47
1,4/1/2020,Barnstable,Bourne,41,41
2,4/1/2020,Barnstable,Brewster,8,8
3,4/1/2020,Barnstable,Chatham,3,3
4,4/1/2020,Barnstable,Dennis,13,13


## Check columns and data types

In [ ]:
def summarize_dataframe(df, name):
    column_info = [
        {
            "Column Name": col,
            "Non-Null Count": df[col].notnull().sum(),
            "Dtype": str(df[col].dtype)
        }
        for col in df.columns
    ]
    return name, column_info

dataframes = {
    "Encounters": encounters,
    "Patients": patients,
    "Payers": payers,
    "Procedures": procedures,
    "COVID_19_Test_to_Treat":COVID_19_Test_to_Treat,
    "Daily_Town_COVID19": Daily_Town_COVID19
}

for name, df in dataframes.items():
    df_name, column_info = summarize_dataframe(df, name)
    print(f"\n{'='*10} {df_name} {'='*10}")
    print(tabulate(column_info, headers="keys", tablefmt="fancy_grid"))



========== Encounters ==========
╒═════════════════════╤══════════════════╤═════════╕
│ Column Name         │   Non-Null Count │ Dtype   │
╞═════════════════════╪══════════════════╪═════════╡
│ Id                  │            27891 │ object  │
├─────────────────────┼──────────────────┼─────────┤
│ START               │            27891 │ object  │
├─────────────────────┼──────────────────┼─────────┤
│ STOP                │            27891 │ object  │
├─────────────────────┼──────────────────┼─────────┤
│ PATIENT             │            27891 │ object  │
├─────────────────────┼──────────────────┼─────────┤
│ ORGANIZATION        │            27891 │ object  │
├─────────────────────┼──────────────────┼─────────┤
│ PAYER               │            27891 │ object  │
├─────────────────────┼──────────────────┼─────────┤
│ ENCOUNTERCLASS      │            27891 │ object  │
├─────────────────────┼──────────────────┼─────────┤
│ CODE                │            27891 │ int64   │
├───────────

# Data Cleaning

## Names Standardization

### Columns Column Names Standardization

To ensure consistency across datasets, I capitalized the columns names (State, County and City) in COVID_19_Test_to_Treat and Daily_Town_COVID19 to be like in the patients columns

as:

   - State → STATE
   - county → COUNTY
   - City → CITY

In [ ]:
COVID_19_Test_to_Treat.rename(columns={'City': 'CITY','State': 'STATE'}, inplace=True)
Daily_Town_COVID19.rename(columns={'County': 'COUNTY'}, inplace=True)

### Standardization Values


In this section, I applied the following changes to ensure consistency across datasets:

1. **County Name Standardization**:
   - In the `patients['COUNTY']` column, I removed the last word "County" from each value to match the values in `Daily_Town_COVID19['COUNTY']`.

2. **City Name Standardization**:
   - In the `COVID_19_Test_to_Treat['CITY']` column, I capitalized only the first letter of each city name (i.e., made the first character uppercase and the rest lowercase) to match the format in `patients['CITY']`.


In [ ]:
# 1. County Name Standardization
patients['COUNTY'] = patients['COUNTY'].str.replace(r'\sCounty$', '', regex=True)

In [ ]:
COVID_19_Test_to_Treat['CITY'] = COVID_19_Test_to_Treat['CITY'].str.capitalize()

## Data Filtering and Selection


In this stage, I refined the datasets to ensure alignment with the primary focus of the analysis: **patients in the state of Massachusetts**, specifically in certain counties and cities. The datasets `COVID_19_Test_to_Treat` and `Daily_Town_COVID19` contain information from multiple states, counties, and cities. However, to match the patient data, which is exclusively related to Massachusetts, I performed the following steps:  

1. **State Filtering**:  
   - Dropped rows for states other than Massachusetts to limit the data to the relevant geographic region and change the abbreviation from MA to Massachusetts.  

2. **County and City Filtering**:  
   - Removed rows for counties and cities that do not align with the patient data. This ensures consistency across datasets and focuses the analysis on the relevant locations.   

### Checking patients data for STATE,COUNTY AND CITY

In [ ]:
patients['STATE'].unique()

array(['Massachusetts'], dtype=object)

In [ ]:
patients['COUNTY'].unique()

array(['Norfolk', 'Suffolk', 'Middlesex', 'Plymouth', 'Essex'],
      dtype=object)

In [ ]:
patients['CITY'].unique()

array(['Quincy', 'Boston', 'Cambridge', 'Somerville', 'Revere',
       'Cohasset', 'Everett', 'Melrose', 'Chelsea', 'Weymouth',
       'Stoneham', 'Hingham', 'Newton', 'Medford', 'Winthrop',
       'Brookline', 'Braintree', 'Lynnfield', 'Hull', 'Scituate',
       'Malden', 'Milton', 'Reading', 'Norwell', 'Winchester', 'Belmont',
       'North Scituate', 'Waltham', 'Watertown'], dtype=object)

### Drop the rows in COVID_19_Test_to_Treat and Daily_Town_COVID19 to match the patients STATE,COUNTY AND CITY and Change STATE values from MA to Massachusetts

#### COVID_19_Test_to_Treat

In this dataset, the states are represented using abbreviations. Since we are focusing on Massachusetts, it is expressed as 'MA'.

In [ ]:
# Dealing with the STATE
COVID_19_Test_to_Treat = COVID_19_Test_to_Treat[COVID_19_Test_to_Treat['STATE'] == 'MA'].reset_index(drop=True)

In [ ]:
# Dealing with the CITY
COVID_19_Test_to_Treat = COVID_19_Test_to_Treat[COVID_19_Test_to_Treat['CITY'].isin(patients['CITY'].unique())].reset_index(drop=True)


In [ ]:
# Dealing with STATE values from MA to Massachusetts
state_map = {'MA': 'Massachusetts'}
COVID_19_Test_to_Treat['STATE'] = COVID_19_Test_to_Treat['STATE'].replace(state_map)

#### Daily_Town_COVID19

In [ ]:
Daily_Town_COVID19 = Daily_Town_COVID19[Daily_Town_COVID19['COUNTY'].isin(patients['COUNTY'].unique())].reset_index(drop=True)

## Check and Handeling Missing Values

In [ ]:
# Function to calculate missing values and percentages
def missing_values_percentage(df, dataset_name):
    missing = df.isnull().sum()
    total = len(df)
    percent = (missing / total) * 100
    return pd.DataFrame({
        'Dataset': dataset_name,
        'Column': df.columns,
        'Missing Count': missing.values,
        'Missing Percentage (%)': percent.values
    })

# Check missing values for all datasets
datasets = {
    "encounters": encounters,
    "patients": patients,
    "payers": payers,
    "procedures": procedures,
    "COVID_19_Test_to_Treat": COVID_19_Test_to_Treat,
    "Daily_Town_COVID19": Daily_Town_COVID19
}

# Concatenate results
results = pd.concat(
    [missing_values_percentage(df, name) for name, df in datasets.items()],
    ignore_index=True
)

# Save or view results
results.sort_values(by=['Dataset', 'Missing Percentage (%)'], ascending=[True, False], inplace=True)
results


,Dataset,Column,Missing Count,Missing Percentage (%)
58,COVID_19_Test_to_Treat,Phone Number,191,99.479167
57,COVID_19_Test_to_Treat,Public Website,182,94.791667
52,COVID_19_Test_to_Treat,Address2,172,89.583333
50,COVID_19_Test_to_Treat,Provider Name,0,0.000000
51,COVID_19_Test_to_Treat,Address1,0,0.000000
...,...,...,...,...
43,procedures,PATIENT,0,0.000000
44,procedures,ENCOUNTER,0,0.000000
45,procedures,CODE,0,0.000000
46,procedures,DESCRIPTION,0,0.000000


*  **encounters.csv**

From results we had 19541  (70.1%) missing values in each columns of REASONCODE and REASONDESCRIPTION. REASONCODE represent diagnosis code from SNOMED-CT, only if this encounter targeted a specific condition and REASONDESCRIPTION represent description of the reason code. 
The diagnosis code from SNOMED-CT refers to a unique identifier used to classify a specific medical condition or diagnosis based on the SNOMED-CT (Systematized Nomenclature of Medicine—Clinical Terms) coding system. So it makes sense that there could be a lot of missing values in the REASONCODE and REASONDESCRIPTION columns, especially when the ReasonCode is only populated in a specific condition. Additionally, it is important to note that there is no key or indicator in the dataset that directly links or suggests a relationship with the REASONCODE. The missing values in REASONCODE and REASONDESCRIPTION are therefore not indicative of data quality issues but reflect the design of the encounters, where only specific conditions result in a populated REASONCODE. The handling of missing values first we fill REASONCODE missing values with -1.0, and then filling the missing values in both REASONDESCRIPTION with ("None").

In [ ]:
# Filling the missing values
encounters['REASONCODE'] = encounters['REASONCODE'].fillna(-1)
encounters['REASONDESCRIPTION'] = encounters['REASONDESCRIPTION'].fillna("None")


*  **procedures.csv**

From results we had 36945  (77.5% )missing values in each columns of REASONCODE and REASONDESCRIPTION. The same situation like what we face with **encounters.csv**.

In [ ]:
# Filling the missing values
procedures['REASONCODE'] = procedures['REASONCODE'].fillna(-1)
procedures['REASONDESCRIPTION'] = procedures['REASONDESCRIPTION'].fillna("None")


* **patients.csv**

From results we had the following missing values in these columns:
 - 820 (84.2%) missing values in DEATHDATE
 - 953 (97.8%) missing values in SUFFIX
 - 588 (60.4%) missing values in MAIDEN
 - 1   (0.1%)  missing values in MARTIAL
 - 142 (14.6%) missing values in ZIP 
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
- **DEATHDATE**: Missing values in this column are expected, as not all patients in the dataset are deceased. The absence of a death date likely indicates that the patient is still alive. Since this columns is an object type handeling this missing by filling empty string (" ").
- **SUFFIX**: This column represents name suffixes, such as PhD, MD, or JD. The high proportion of missing values is reasonable, as suffixes are not commonly used by most individuals. We will handle this missing that filling empty string (" ").
- **MAIDEN**: Missing values in this column are logical, particularly for male patients, as they typically do not change their last name after marriage. Handle this missing by filling empty string (" ").
- **MARITAL**: With only 0.1% of values missing in this column, the impact on the dataset is negligible. These rows can be dropped without significantly affecting the analysis.
- **ZIP**: Missing ZIP codes could result from data entry errors or patients not providing this information. For handeling the missing values we will change the column type changed from float64 to string and then filling it with "Unknown".

In [ ]:
# Drop the row where the MARITAL is missing
patients.dropna(subset=['MARITAL'], inplace=True)
# Convert ZIP from float64 to object
patients['ZIP'] = patients['ZIP'].fillna('Unknown').astype(str)
# Fill NaN values with empty string
patients.fillna(" ", inplace=True)

* **payers.csv**

From results the missing values are found in the last row, and upon review, it is logical for the following columns to have missing values ADDRESS, CITY, STATE_HEADQUARTERED, ZIP, and PHONE as the entry corresponds to the name "No Insurance". To handle the missing values in the columns ADDRESS, CITY, STATE_HEADQUARTERED, and PHONE, we will fill them with an empty string (" "). For the column ZIP, we will first change its data type from float64 to object and then fill the missing values with an empty string (" ").

In [ ]:
# Convert ZIP from float64 to object
payers['ZIP'] = payers['ZIP'].fillna('Unknown').astype(str)
# Fill NaN values with empty string
payers.fillna("", inplace=True)

* **COVID_19_Test_to_Treat**

From results the missing values are Address2 (89.58% missing),Public Website (94.79% missing) and Phone Number (99.47% missing). The best action is drop the columns, it likely adds negligible value to the dataset.

In [ ]:
COVID_19_Test_to_Treat.drop(columns = ['Address2','Public Website', 'Phone Number'], inplace = True)

* **Daily_Town_COVID19**

From results there are no missing values

## Data Profiling Report

In [ ]:
# Create the "Data Profiling Report"
output_folder = "Data Profiling Report"
os.makedirs(output_folder, exist_ok=True)
# Generate and save individual reports inside the folder

profile_encounters = ProfileReport(encounters, title="Encounters Profiling Report", explorative=True)
profile_encounters.to_file(os.path.join(output_folder, "encounters_report.html"))

profile_patients = ProfileReport(patients, title="Patients Profiling Report", explorative=True)
profile_patients.to_file(os.path.join(output_folder, "patients_report.html"))

profile_payers = ProfileReport(payers, title="Payers Profiling Report", explorative=True)
profile_payers.to_file(os.path.join(output_folder, "payers_report.html"))

profile_procedures = ProfileReport(procedures, title="Procedures Profiling Report", explorative=True)
profile_procedures.to_file(os.path.join(output_folder, "procedures_report.html"))

profile_COVID_19_Test_to_Treat = ProfileReport(COVID_19_Test_to_Treat, title="COVID_19_Test_to_Treat Profiling Report", explorative=True)
profile_COVID_19_Test_to_Treat.to_file(os.path.join(output_folder, "COVID_19_Test_to_Treat_report.html"))

profile_Daily_Town_COVID19 = ProfileReport(Daily_Town_COVID19, title="Daily_Town_COVID19 Profiling Report", explorative=True)
profile_Daily_Town_COVID19.to_file(os.path.join(output_folder, "Daily_Town_COVID19_report.html"))

# Outliers Detection:
Identify and investigate potential outliers using IQR methods, Box Plots, and Z-Scores.

The primary datasets for analysis are encounters, patients,procedures, COVID_19_Test_to_Treat and Daily_Town_COVID19. The other file payers, serve to provide additional context and clarify the meaning of columns in the main datasets. This file are not part of the core analysis but are used for interpretation purposes.

## IQR and Z-Score

### IQR (Interquartile Range) Method:

The IQR method is based on the spread of the middle 50% of the data. It defines the range between the 25th percentile (Q1) and the 75th percentile (Q3), and is used to detect outliers.

How it works:
   - **IQR**: The difference between Q3 (75th percentile) and Q1 (25th percentile).
   - **Outlier Detection**: Any data point that lies outside the following bounds is considered an outlier:
       - **Lower Bound**: 
         $$ \text{Lower Bound} = Q1 - 1.5 \times IQR $$
       - **Upper Bound**: 
         $$ \text{Upper Bound} = Q3 + 1.5 \times IQR $$
   - If a data point is below the lower bound or above the upper bound, it is flagged as an outlier.

Interpretation: The IQR method focuses on the spread of the central data and detects outliers as values significantly lower or higher than the majority of the data.


### Z-Scores:

The z-score measures how many standard deviations a data point is away from the mean of the dataset.

How it works:
   - The z-score is calculated using the formula:
       $$ Z = \frac{X - \mu}{\sigma} $$
   - where:
       - X is the data point.
       - μ is the mean of the data.
       - σ is the standard deviation of the data.
   - A z-score indicates how far a data point is from the mean in terms of standard deviations.
   - Typically, a z-score above 3 or below -3 is considered an outlier (meaning the data point is more than 3 standard deviations away from the mean).

Interpretation: A large z-score (either positive or negative) indicates that the data point is far from the mean, suggesting it may be an outlier.



In [ ]:
# Function to detect outliers using IQR
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] < lower_bound) | (data[column] > upper_bound)]

# Function to detect outliers using z-score
def detect_outliers_zscore(data, column):
    threshold = 3  # Define the threshold for Z-score outliers (e.g., 3 standard deviations)
    mean = data[column].mean()
    std = data[column].std()
    z_scores = (data[column] - mean) / std
    return data[(z_scores > threshold) | (z_scores < -threshold)]

# Apply outlier detection to all datasets and compile results into a table
def analyze_outliers(data_dict):
    outlier_results = []

    for dataset_name, data in data_dict.items():
        for column in data.select_dtypes(include=[np.number]).columns:
            
            # IQR Method
            iqr_outliers = detect_outliers_iqr(data, column)
            iqr_count = len(iqr_outliers)
            iqr_percentage = (iqr_count / data.shape[0]) * 100  # Calculate percentage of IQR outliers
            
            # Z-score Method
            zscore_outliers = detect_outliers_zscore(data, column)
            zscore_count = len(zscore_outliers)
            
            # Append results to the table
            outlier_results.append({
                'Dataset': dataset_name,
                'Column': column,
                'IQR Outliers': iqr_count,
                'IQR Percentage': f"{iqr_percentage:.2f}%",
                'Z-Score Outliers': zscore_count,
            })
    
    # Convert the results to a DataFrame for easy display
    outlier_df = pd.DataFrame(outlier_results)
    return outlier_df

# Example: Analyze outliers for all datasets
data_dict = {
    'encounters': encounters,
    'patients': patients,
    'procedures': procedures,
    "COVID_19_Test_to_Treat":COVID_19_Test_to_Treat,
    "Daily_Town_COVID19": Daily_Town_COVID19
}

outlier_df = analyze_outliers(data_dict)
outlier_df


,Dataset,Column,IQR Outliers,IQR Percentage,Z-Score Outliers
0,encounters,CODE,0,0.00%,0
1,encounters,BASE_ENCOUNTER_COST,0,0.00%,0
2,encounters,TOTAL_CLAIM_COST,5316,19.06%,535
3,encounters,PAYER_COVERAGE,5035,18.05%,512
4,encounters,REASONCODE,2401,8.61%,56
5,patients,LAT,35,3.60%,3
6,patients,LON,45,4.62%,16
7,procedures,CODE,6670,13.98%,144
8,procedures,BASE_COST,9174,19.23%,1253
9,procedures,REASONCODE,10756,22.55%,53


In [ ]:
outlier_df.to_csv('outliers before.csv')

We will exclude the encounters.CODE column due to IQR and Z-Score are 0, indicating no variation in the data. Additionally, both the encounters.CODE and procedures.CODE columns will be excluded because they represent standardized codes tied to real-world medical descriptions. These codes, while rare, are valid and do not signify errors or anomalies, meaning that outliers in these columns are not meaningful.

We will also exclude the encounters.REASONCODE and procedures.REASONCODE columns. These columns represent specific codes related to medical reasons and procedures, and the presence of outliers in these columns is not indicative of errors, but rather rare cases that are part of the valid medical categorization.

Finally, the encounters.BASE_ENCOUNTER_COST column will be excluded due to IQR and Z-Score are 0, indicating no variability in the data, making it unsuitable for outlier analysis.

## Box Plots

Box plots are a graphical way to display the distribution of data and identify potential outliers.

How it works:
   - **Box**: Represents the interquartile range (IQR), which is the range between the 25th percentile (Q1) and the 75th percentile (Q3) of the data.
   - **Outlier Detection**: The lines extending from the box. They typically represent the range of the data within 1.5 times the IQR from Q1 and Q3.
   - **Outliers**: Data points are below the lower bound or above the upper bound (i.e., beyond 1.5 * IQR from Q1 or Q3) are considered outliers and are often plotted as individual points.


### Box Plot of TOTAL_CLAIM_COST (encounter)

In [ ]:
# Calculate IQR values
Q1 = encounters['TOTAL_CLAIM_COST'].quantile(0.25)
Q3 = encounters['TOTAL_CLAIM_COST'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR
min_value = encounters['TOTAL_CLAIM_COST'].min()
max_value = encounters['TOTAL_CLAIM_COST'].max()
median_value = encounters['TOTAL_CLAIM_COST'].median()

# Create a subplot with 2 rows: one for the box plot, one for the table
fig_BT = make_subplots(
    rows=2, cols=1,
    row_heights=[0.7, 0.3],  # Allocate space for the graph and table
    vertical_spacing=0.1,
    specs=[[{"type": "xy"}], [{"type": "table"}]]  # First row: xy plot, second row: table
)

# Add the box plot layer
fig_BT.add_trace(
    go.Box(
        y=encounters['TOTAL_CLAIM_COST'],
        name='TOTAL_CLAIM_COST',
        boxpoints='outliers',  # Show outliers as individual points
        marker=dict(color='#1f77b4', outliercolor='#ff7f0e', size=6),  # Outliers in a different color
        line=dict(color='#1f77b4')
    ),
    row=1, col=1
)

# Add IQR lines
fig_BT.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],  # Extend the line horizontally
        y=[Q1, Q1],
        mode='lines',
        line=dict(color='green', dash='dash'),
        name='Q1 (25th Percentile)'
    ),
    row=1, col=1
)
fig_BT.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],
        y=[Q3, Q3],
        mode='lines',
        line=dict(color='blue', dash='dash'),
        name='Q3 (75th Percentile)'
    ),
    row=1, col=1
)

# Add the table with statistics
fig_BT.add_trace(
    go.Table(
        header=dict(
            values=["Statistic", "Value"],
            fill_color="lightgrey",
            align="center",
            font=dict(size=12, color="black")
        ),
        cells=dict(
            values=[
                ["Min", "Max", "Median", "Q1 (25th Percentile)", "Q3 (75th Percentile)", "Lower Limit", "Upper Limit", "IQR"],
                [min_value, max_value, median_value, Q1, Q3, lower_limit, upper_limit, IQR]
            ],
            fill_color="white",
            align="center",
            font=dict(size=11, color="black")
        )
    ),
    row=2, col=1
)

# Update layout for better visibility
fig_BT.update_layout(
    title='Box Plot of TOTAL_CLAIM_COST with Statistics Table (encounter)',
    xaxis_title='',
    yaxis_title='TOTAL_CLAIM_COST',
    yaxis=dict(range=[lower_limit, upper_limit + 500]),  # Slightly extend the range for clarity
    height=880,  # Adjust height to accommodate both graph and table
    width=800,
    showlegend=True,  # Show legend for IQR lines
    xaxis=dict(showticklabels=False)
)
# Show the plot
fig_BT.show()


### Box Plot of PAYER_COVERAGE (encounter)

In [ ]:
# Calculate IQR values
Q1 = encounters['PAYER_COVERAGE'].quantile(0.25)
Q3 = encounters['PAYER_COVERAGE'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR
min_value = encounters['PAYER_COVERAGE'].min()
max_value = encounters['PAYER_COVERAGE'].max()
median_value = encounters['PAYER_COVERAGE'].median()

# Create a subplot with 2 rows: one for the box plot, one for the table
fig_BP = make_subplots(
    rows=2, cols=1,
    row_heights=[0.7, 0.3],  # Allocate space for the graph and table
    vertical_spacing=0.1,
    specs=[[{"type": "xy"}], [{"type": "table"}]]  # First row: xy plot, second row: table
)

# Add the box plot layer
fig_BP.add_trace(
    go.Box(
        y=encounters['PAYER_COVERAGE'],
        name='PAYER_COVERAGE',
        boxpoints='outliers',  # Show outliers as individual points
        marker=dict(color='#1f77b4', outliercolor='#ff7f0e', size=6),  # Outliers in a different color
        line=dict(color='#1f77b4')
    ),
    row=1, col=1
)

# Add IQR lines
fig_BP.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],  # Extend the line horizontally
        y=[Q1, Q1],
        mode='lines',
        line=dict(color='green', dash='dash'),
        name='Q1 (25th Percentile)'
    ),
    row=1, col=1
)
fig_BP.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],
        y=[Q3, Q3],
        mode='lines',
        line=dict(color='blue', dash='dash'),
        name='Q3 (75th Percentile)'
    ),
    row=1, col=1
)

# Add the table with statistics
fig_BP.add_trace(
    go.Table(
        header=dict(
            values=["Statistic", "Value"],
            fill_color="lightgrey",
            align="center",
            font=dict(size=12, color="black")
        ),
        cells=dict(
            values=[
                ["Min", "Max", "Median", "Q1 (25th Percentile)", "Q3 (75th Percentile)", "Lower Limit", "Upper Limit", "IQR"],
                [min_value, max_value, median_value, Q1, Q3, lower_limit, upper_limit, IQR]
            ],
            fill_color="white",
            align="center",
            font=dict(size=11, color="black")
        )
    ),
    row=2, col=1
)

# Update layout for better visibility
fig_BP.update_layout(
    title='Box Plot of PAYER_COVERAGE with Statistics Table (encounter)',
    xaxis_title='',
    yaxis_title='PAYER_COVERAGE',
    yaxis=dict(range=[lower_limit, upper_limit + 500]),  # Slightly extend the range for clarity
    height=880,  # Adjust height to accommodate both graph and table
    width=800,
    showlegend=True,  # Show legend for IQR lines
    xaxis=dict(showticklabels=False)
)

# Show the plot
fig_BP.show()


### Box Plot of LAT with Statistics (patients)

In [ ]:
# Calculate IQR values for LAT
Q1 = patients['LAT'].quantile(0.25)
Q3 = patients['LAT'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR
min_value = patients['LAT'].min()
max_value = patients['LAT'].max()
median_value = patients['LAT'].median()

# Create a subplot with 2 rows: one for the box plot, one for the table
fig_BLAT = make_subplots(
    rows=2, cols=1,
    row_heights=[0.7, 0.3],  # Allocate space for the graph and table
    vertical_spacing=0.1,
    specs=[[{"type": "xy"}], [{"type": "table"}]]  # First row: xy plot, second row: table
)

# Add the box plot layer for LAT
fig_BLAT.add_trace(
    go.Box(
        y=patients['LAT'],
        name='LAT',
        boxpoints='outliers',  # Show outliers as individual points
        marker=dict(color='#1f77b4', outliercolor='#ff7f0e', size=6),  # Outliers in a different color
        line=dict(color='#1f77b4')
    ),
    row=1, col=1
)

# Add Q1 and Q3 lines for LAT
fig_BLAT.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],  # Extend the line horizontally
        y=[Q1, Q1],
        mode='lines',
        line=dict(color='green', dash='dash'),
        name='Q1 (25th Percentile)'
    ),
    row=1, col=1
)
fig_BLAT.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],
        y=[Q3, Q3],
        mode='lines',
        line=dict(color='blue', dash='dash'),
        name='Q3 (75th Percentile)'
    ),
    row=1, col=1
)

# Add the table with statistics
fig_BLAT.add_trace(
    go.Table(
        header=dict(
            values=["Statistic", "Value"],
            fill_color="lightgrey",
            align="center",
            font=dict(size=12, color="black")
        ),
        cells=dict(
            values=[
                ["Min", "Max", "Median", "Q1 (25th Percentile)", "Q3 (75th Percentile)", "Lower Limit", "Upper Limit", "IQR"],
                [min_value, max_value, median_value, Q1, Q3, lower_limit, upper_limit, IQR]
            ],
            fill_color="white",
            align="center",
            font=dict(size=11, color="black")
        )
    ),
    row=2, col=1
)

# Update layout for better visibility with y-axis range focused between 40 and 50
fig_BLAT.update_layout(
    title='Box Plot of LAT with Statistics Table (patients)',
    xaxis_title='',
    yaxis_title='LAT',
    yaxis=dict(range=[42.2, 42.5]),  # Set the y-axis range between 40 and 50
    height=880,  # Adjust height to accommodate both graph and table
    width=800,
    showlegend=True,  # Show legend for Q1 and Q3 lines
    xaxis=dict(showticklabels=False)
)

# Show the plot
fig_BLAT.show()


### Box Plot of LON with Statistics (patients)

In [ ]:
# Calculate IQR values for LON
Q1 = patients['LON'].quantile(0.25)
Q3 = patients['LON'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR
min_value = patients['LON'].min()
max_value = patients['LON'].max()
median_value = patients['LON'].median()

# Create a subplot with 2 rows: one for the box plot, one for the table
fig_BLON = make_subplots(
    rows=2, cols=1,
    row_heights=[0.7, 0.3],  # Allocate space for the graph and table
    vertical_spacing=0.1,
    specs=[[{"type": "xy"}], [{"type": "table"}]]  # First row: xy plot, second row: table
)

# Add the box plot layer for LON
fig_BLON.add_trace(
    go.Box(
        y=patients['LON'],
        name='LON',
        boxpoints='outliers',  # Show outliers as individual points
        marker=dict(color='#1f77b4', outliercolor='#ff7f0e', size=6),  # Outliers in a different color
        line=dict(color='#1f77b4')
    ),
    row=1, col=1
)

# Add IQR lines for LON
fig_BLON.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],  # Extend the line horizontally
        y=[Q1, Q1],
        mode='lines',
        line=dict(color='green', dash='dash'),
        name='Q1 (25th Percentile)'
    ),
    row=1, col=1
)
fig_BLON.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],
        y=[Q3, Q3],
        mode='lines',
        line=dict(color='blue', dash='dash'),
        name='Q3 (75th Percentile)'
    ),
    row=1, col=1
)

# Add the table with statistics
fig_BLON.add_trace(
    go.Table(
        header=dict(
            values=["Statistic", "Value"],
            fill_color="lightgrey",
            align="center",
            font=dict(size=12, color="black")
        ),
        cells=dict(
            values=[
                ["Min", "Max", "Median", "Q1 (25th Percentile)", "Q3 (75th Percentile)", "Lower Limit", "Upper Limit", "IQR"],
                [min_value, max_value, median_value, Q1, Q3, lower_limit, upper_limit, IQR]
            ],
            fill_color="white",
            align="center",
            font=dict(size=11, color="black")
        )
    ),
    row=2, col=1
)

# Update layout for better visibility
fig_BLON.update_layout(
    title='Box Plot of LON with Statistics Table (patients)',
    xaxis_title='',
    yaxis_title='LON',
    yaxis=dict(range=[-71.2,-70.8]),  # Slightly extend the range for clarity
    height=880,  # Adjust height to accommodate both graph and table
    width=800,
    showlegend=True,  # Show legend for IQR lines
    xaxis=dict(showticklabels=False)
)

# Show the plot
fig_BLON.show()


### Box Plot of BASE_COST (procedures)

In [ ]:
# Calculate IQR values for BASE_COST
Q1 = procedures['BASE_COST'].quantile(0.25)
Q3 = procedures['BASE_COST'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR
min_value = procedures['BASE_COST'].min()
max_value = procedures['BASE_COST'].max()
median_value = procedures['BASE_COST'].median()

# Create a subplot with 2 rows: one for the box plot, one for the table
fig_BB = make_subplots(
    rows=2, cols=1,
    row_heights=[0.7, 0.3],  # Allocate space for the graph and table
    vertical_spacing=0.1,
    specs=[[{"type": "xy"}], [{"type": "table"}]]  # First row: xy plot, second row: table
)

# Add the box plot layer for BASE_COST
fig_BB.add_trace(
    go.Box(
        y=procedures['BASE_COST'],
        name='BASE_COST',
        boxpoints='outliers',  # Show outliers as individual points
        marker=dict(color='#1f77b4', outliercolor='#ff7f0e', size=6),  # Outliers in a different color
        line=dict(color='#1f77b4')
    ),
    row=1, col=1
)

# Add IQR lines for BASE_COST
fig_BB.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],  # Extend the line horizontally
        y=[Q1, Q1],
        mode='lines',
        line=dict(color='green', dash='dash'),
        name='Q1 (25th Percentile)'
    ),
    row=1, col=1
)
fig_BB.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],
        y=[Q3, Q3],
        mode='lines',
        line=dict(color='blue', dash='dash'),
        name='Q3 (75th Percentile)'
    ),
    row=1, col=1
)

# Add the table with statistics
fig_BB.add_trace(
    go.Table(
        header=dict(
            values=["Statistic", "Value"],
            fill_color="lightgrey",
            align="center",
            font=dict(size=12, color="black")
        ),
        cells=dict(
            values=[
                ["Min", "Max", "Median", "Q1 (25th Percentile)", "Q3 (75th Percentile)", "Lower Limit", "Upper Limit", "IQR"],
                [min_value, max_value, median_value, Q1, Q3, lower_limit, upper_limit, IQR]
            ],
            fill_color="white",
            align="center",
            font=dict(size=11, color="black")
        )
    ),
    row=2, col=1
)

# Update layout for better visibility
fig_BB.update_layout(
    title='Box Plot of BASE_COST with Statistics Table (procedures)',
    xaxis_title='',
    yaxis_title='BASE_COST',
    yaxis=dict(range=[lower_limit, upper_limit + 500]),  # Slightly extend the range for clarity
    height=880,  # Adjust height to accommodate both graph and table
    width=800,
    showlegend=True,  # Show legend for IQR lines
    xaxis=dict(showticklabels=False)
)

# Show the plot
fig_BB.show()


### Box Plot of covid_cases_cumulative (Daily_Town_COVID19)

In [ ]:
# Calculate IQR values
Q1 = Daily_Town_COVID19['covid_cases_cumulative'].quantile(0.25)
Q3 = Daily_Town_COVID19['covid_cases_cumulative'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR
min_value = Daily_Town_COVID19['covid_cases_cumulative'].min()
max_value = Daily_Town_COVID19['covid_cases_cumulative'].max()
median_value = Daily_Town_COVID19['covid_cases_cumulative'].median()

# Create a subplot with 2 rows: one for the box plot, one for the table
fig_BC = make_subplots(
    rows=2, cols=1,
    row_heights=[0.7, 0.3],  # Allocate space for the graph and table
    vertical_spacing=0.1,
    specs=[[{"type": "xy"}], [{"type": "table"}]]  # First row: xy plot, second row: table
)

# Add the box plot layer
fig_BC.add_trace(
    go.Box(
        y=Daily_Town_COVID19['covid_cases_cumulative'],
        name='covid_cases_cumulative',
        boxpoints='outliers',  # Show outliers as individual points
        marker=dict(color='#1f77b4', outliercolor='#ff7f0e', size=6),  # Outliers in a different color
        line=dict(color='#1f77b4')
    ),
    row=1, col=1
)

# Add IQR lines
fig_BC.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],  # Extend the line horizontally
        y=[Q1, Q1],
        mode='lines',
        line=dict(color='green', dash='dash'),
        name='Q1 (25th Percentile)'
    ),
    row=1, col=1
)
fig_BC.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],
        y=[Q3, Q3],
        mode='lines',
        line=dict(color='blue', dash='dash'),
        name='Q3 (75th Percentile)'
    ),
    row=1, col=1
)

# Add the table with statistics
fig_BC.add_trace(
    go.Table(
        header=dict(
            values=["Statistic", "Value"],
            fill_color="lightgrey",
            align="center",
            font=dict(size=12, color="black")
        ),
        cells=dict(
            values=[
                ["Min", "Max", "Median", "Q1 (25th Percentile)", "Q3 (75th Percentile)", "Lower Limit", "Upper Limit", "IQR"],
                [min_value, max_value, median_value, Q1, Q3, lower_limit, upper_limit, IQR]
            ],
            fill_color="white",
            align="center",
            font=dict(size=11, color="black")
        )
    ),
    row=2, col=1
)

# Update layout for better visibility
fig_BC.update_layout(
    title='Box Plot of covid_cases_cumulative with Statistics Table (Daily_Town_COVID19)',
    xaxis_title='',
    yaxis_title='covid_cases_cumulative',
    yaxis=dict(range=[lower_limit, upper_limit + 500]),  # Slightly extend the range for clarity
    height=880,  # Adjust height to accommodate both graph and table
    width=800,
    showlegend=True,  # Show legend for IQR lines
    xaxis=dict(showticklabels=False)
)

# Show the plot
fig_BC.show()


### Box Plot of daily_cases (Daily_Town_COVID19)

In [ ]:
# Calculate IQR values
Q1 = Daily_Town_COVID19['daily_cases'].quantile(0.25)
Q3 = Daily_Town_COVID19['daily_cases'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR
min_value = Daily_Town_COVID19['daily_cases'].min()
max_value = Daily_Town_COVID19['daily_cases'].max()
median_value = Daily_Town_COVID19['daily_cases'].median()

# Create a subplot with 2 rows: one for the box plot, one for the table
fig_BD = make_subplots(
    rows=2, cols=1,
    row_heights=[0.7, 0.3],  # Allocate space for the graph and table
    vertical_spacing=0.1,
    specs=[[{"type": "xy"}], [{"type": "table"}]]  # First row: xy plot, second row: table
)

# Add the box plot layer
fig_BD.add_trace(
    go.Box(
        y=Daily_Town_COVID19['daily_cases'],
        name='daily_cases',
        boxpoints='outliers',  # Show outliers as individual points
        marker=dict(color='#1f77b4', outliercolor='#ff7f0e', size=6),  # Outliers in a different color
        line=dict(color='#1f77b4')
    ),
    row=1, col=1
)

# Add IQR lines
fig_BD.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],  # Extend the line horizontally
        y=[Q1, Q1],
        mode='lines',
        line=dict(color='green', dash='dash'),
        name='Q1 (25th Percentile)'
    ),
    row=1, col=1
)
fig_BD.add_trace(
    go.Scatter(
        x=[-0.5, 0.5],
        y=[Q3, Q3],
        mode='lines',
        line=dict(color='blue', dash='dash'),
        name='Q3 (75th Percentile)'
    ),
    row=1, col=1
)

# Add the table with statistics
fig_BD.add_trace(
    go.Table(
        header=dict(
            values=["Statistic", "Value"],
            fill_color="lightgrey",
            align="center",
            font=dict(size=12, color="black")
        ),
        cells=dict(
            values=[
                ["Min", "Max", "Median", "Q1 (25th Percentile)", "Q3 (75th Percentile)", "Lower Limit", "Upper Limit", "IQR"],
                [min_value, max_value, median_value, Q1, Q3, lower_limit, upper_limit, IQR]
            ],
            fill_color="white",
            align="center",
            font=dict(size=11, color="black")
        )
    ),
    row=2, col=1
)

# Update layout for better visibility
fig_BD.update_layout(
    title='Box Plot of daily_cases with Statistics Table (Daily_Town_COVID19)',
    xaxis_title='',
    yaxis_title='daily_cases',
    yaxis=dict(range=[lower_limit - 5, upper_limit + 50]),  # Slightly extend the range for clarity
    height=880,  # Adjust height to accommodate both graph and table
    width=800,
    showlegend=True,  # Show legend for IQR lines
    xaxis=dict(showticklabels=False)
)

# Show the plot
fig_BD.show()


### Explore The Outliers

#### encounters (TOTAL_CLAIM_COST)

In [ ]:
upper_limit = 3317.4550

# Filter out the rows where TOTAL_CLAIM_COST is above the upper limit and below the upper limit
outliers_above_upper = encounters[encounters['TOTAL_CLAIM_COST'] > upper_limit]
non_outliers = encounters[encounters['TOTAL_CLAIM_COST'] <= upper_limit]

# Get the unique combinations of DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST for outliers and non-outliers
outlier_combinations = set(zip(outliers_above_upper['DESCRIPTION'], 
                                outliers_above_upper['REASONDESCRIPTION'], 
                                outliers_above_upper['BASE_ENCOUNTER_COST']))
non_outlier_combinations = set(zip(non_outliers['DESCRIPTION'], 
                                   non_outliers['REASONDESCRIPTION'], 
                                   non_outliers['BASE_ENCOUNTER_COST']))

# Find common combinations between the outliers and non-outliers based on DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST
common_combinations = outlier_combinations.intersection(non_outlier_combinations)

# Create a DataFrame for common combinations
if common_combinations:
    common_combinations_df = pd.DataFrame(list(common_combinations), 
                                          columns=['DESCRIPTION', 'REASONDESCRIPTION', 'BASE_ENCOUNTER_COST'])
    print("Common combinations between outliers and non-outliers:")
    print(common_combinations_df.to_string(index=False))  # Print the table without index
else:
    print("No common combinations between outliers and non-outliers.")

# Convert common combinations to a list of tuples
common_combinations_list = list(common_combinations)

# Filter the entire dataset (outliers and non-outliers) to check if the combination matches
matching_rows_whole_data = encounters[encounters.apply(
    lambda row: (row['DESCRIPTION'], row['REASONDESCRIPTION'], row['BASE_ENCOUNTER_COST']) in common_combinations_list, axis=1)]


Common combinations between outliers and non-outliers:
                                                           DESCRIPTION                           REASONDESCRIPTION  BASE_ENCOUNTER_COST
                                                 Encounter for symptom                        Sinusitis (disorder)                85.55
Periodic reevaluation and management of healthy individual (procedure)                                        None               146.18
                                                   Emergency Encounter                                        None               146.18
                                            Consultation for treatment                                        None               142.58
                                  Emergency room admission (procedure)                                        None               146.18
                                                 Encounter for problem Chronic congestive heart failure (disorder)               

In [ ]:
# Calculate the total number of matching rows
total_matching_rows = len(matching_rows_whole_data)

# Calculate the percentage of matching rows in the entire dataset
percentage_matching_whole_data = (total_matching_rows / len(encounters)) * 100
# Print the results
print(f"Total number of rows with common DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST: {total_matching_rows} out of {encounters.shape[0]}")
print(f"Percentage of rows in the entire dataset with common DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST: {percentage_matching_whole_data:.2f}%")

Total number of rows with common DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST: 19551 out of 27891
Percentage of rows in the entire dataset with common DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST: 70.10%


Identifies rows where TOTAL_CLAIM_COST is above a specified upper limit (outliers_above_upper) and below or equal to the limit (non_outliers).

Extracts unique combinations of DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST for both outliers and non-outliers.

Finds common combinations between outliers and non-outliers.

Filters the entire dataset to identify rows matching the common combinations.

Calculates and reports the total number and percentage of rows in the dataset with these common combinations.

In [ ]:
upper_limit = 3317.4550

# Filter out the rows where TOTAL_CLAIM_COST is above the upper limit and below or equal to the upper limit
outliers_above_upper = encounters[encounters['TOTAL_CLAIM_COST'] > upper_limit]
non_outliers = encounters[encounters['TOTAL_CLAIM_COST'] <= upper_limit]

# Get the unique combinations of DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST for outliers and non-outliers
outlier_combinations = set(zip(outliers_above_upper['DESCRIPTION'], 
                                outliers_above_upper['REASONDESCRIPTION'], 
                                outliers_above_upper['BASE_ENCOUNTER_COST']))
non_outlier_combinations = set(zip(non_outliers['DESCRIPTION'], 
                                   non_outliers['REASONDESCRIPTION'], 
                                   non_outliers['BASE_ENCOUNTER_COST']))

# Find common combinations between the outliers and non-outliers
common_combinations = outlier_combinations.intersection(non_outlier_combinations)

# Convert common combinations to a list of tuples
common_combinations_list = list(common_combinations)

# Filter the entire dataset to retrieve rows with matching combinations
matching_rows = encounters[encounters.apply(
    lambda row: (row['DESCRIPTION'], row['REASONDESCRIPTION'], row['BASE_ENCOUNTER_COST']) in common_combinations_list, axis=1)]

# Sort the matching rows by DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST
matching_rows_sorted_TOTAL_CLAIM_COST = matching_rows.sort_values(by=['DESCRIPTION', 'REASONDESCRIPTION', 'BASE_ENCOUNTER_COST'])

# Display the organized matching rows
print(f"Total number of matching rows: {len(matching_rows_sorted_TOTAL_CLAIM_COST)}")
print(matching_rows_sorted_TOTAL_CLAIM_COST)


Total number of matching rows: 19551
                                         Id                 START  \
561    6a81d820-f2cc-246d-0822-218d110cabae  2011-06-04T12:43:08Z   
810    3187b136-f844-4b93-3ae4-bdaeec2aa132  2011-08-02T13:23:30Z   
3670   42b7c94b-71a2-1b0e-c132-646f52e35fa4  2013-02-12T03:21:34Z   
3892   9cd78069-e461-2465-b4a6-55b11ce81fca  2013-03-16T15:15:55Z   
4212   83a97735-8d65-c2e9-dda1-2a0800d74f8c  2013-04-26T04:13:21Z   
...                                     ...                   ...   
27861  e050c331-4830-4cb9-fb84-14ee7a817901  2022-01-24T22:04:02Z   
27863  5be73cfd-3631-ba15-1a3d-55fef8aba3c5  2022-01-25T00:21:50Z   
27873  2caa309f-3f07-b3ac-de74-851cd7fdee17  2022-01-26T08:24:25Z   
27884  2de8a9d4-48a6-af90-bfad-2c7acd523054  2022-01-28T23:41:33Z   
27887  07710480-9d6b-9c9b-87c3-c1d54df4069d  2022-01-29T20:12:53Z   

                       STOP                               PATIENT  \
561    2011-06-05T13:23:30Z  814f74d5-6a86-0d84-915d-ea8696834809

In [ ]:
upper_limit = 3317.4550

# Filter the rows where TOTAL_CLAIM_COST is above the upper limit
outliers_above_upper = encounters[encounters['TOTAL_CLAIM_COST'] > upper_limit]

# Get the unique combinations of DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST for the outliers
outlier_combinations = set(zip(outliers_above_upper['DESCRIPTION'], 
                                outliers_above_upper['REASONDESCRIPTION'], 
                                outliers_above_upper['BASE_ENCOUNTER_COST']))

# Create a DataFrame for the outlier combinations
if outlier_combinations:
    outlier_combinations_df = pd.DataFrame(list(outlier_combinations), 
                                           columns=['DESCRIPTION', 'REASONDESCRIPTION', 'BASE_ENCOUNTER_COST'])
    print("Combinations for rows where TOTAL_CLAIM_COST is above the upper limit:")
    print(outlier_combinations_df.to_string(index=False))  # Print the table without index
else:
    print("No combinations for rows where TOTAL_CLAIM_COST is above the upper limit.")


Combinations for rows where TOTAL_CLAIM_COST is above the upper limit:
                                                           DESCRIPTION                                                     REASONDESCRIPTION  BASE_ENCOUNTER_COST
                                                 Encounter for symptom                                                  Sinusitis (disorder)                85.55
                                                 Encounter for problem                                     Suspected lung cancer (situation)                85.55
                                       Non-urgent orthopedic admission                                  Injury of anterior cruciate ligament               146.18
Periodic reevaluation and management of healthy individual (procedure)                                                                  None               146.18
                                       Non-urgent orthopedic admission                          Injury of medial collat

In [ ]:
# Calculate the total number of outlier rows
total_outlier_rows = len(outliers_above_upper)

# Calculate the percentage of outlier rows in the entire dataset
percentage_outliers = (total_outlier_rows / len(encounters)) * 100
# Print the results
print(f"Total number of rows with TOTAL_CLAIM_COST above the upper limit: {total_outlier_rows} out of {encounters.shape[0]}")
print(f"Percentage of rows in the entire dataset with TOTAL_CLAIM_COST above the upper limit: {percentage_outliers:.2f}%")

Total number of rows with TOTAL_CLAIM_COST above the upper limit: 5316 out of 27891
Percentage of rows in the entire dataset with TOTAL_CLAIM_COST above the upper limit: 19.06%


Identifies rows where TOTAL_CLAIM_COST exceeds a specified upper limit (outliers_above_upper).

Extracts unique combinations of DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST for these outliers.

Reports the total number and percentage of outlier rows in the dataset is equal to the IQR outliers and percentage.

##### Summary:  

Through the exploration of rows based on the `TOTAL_CLAIM_COST`, we identified and analyzed outliers exceeding a specified upper limit. These analyses included:  

1. **Comparing Outliers and Non-Outliers**:  
   - Unique combinations of `DESCRIPTION`, `REASONDESCRIPTION`, and `BASE_ENCOUNTER_COST` were extracted for both outliers and non-outliers.  
   - Common patterns between these groups were identified, and their prevalence in the entire dataset was calculated.  

2. **Focusing on Outliers Alone**:  
   - Unique combinations for outliers were examined separately to determine their characteristics.  
   - The proportion of outlier rows in the dataset was assessed, providing insight into the scale of deviation.  

##### Best Action:  
Based on this analysis, **the most suitable action to address these outliers is to drop them**. This decision ensures that extreme values, which may distort statistical models or lead to inaccurate insights, are effectively managed while preserving the integrity of the dataset for further analysis.

#### encounters (PAYER_COVERAGE)

In [ ]:
upper_limit = 389.4250

# Filter out the rows where TOTAL_CLAIM_COST is above the upper limit and below the upper limit
outliers_above_upper = encounters[encounters['PAYER_COVERAGE'] > upper_limit]
non_outliers = encounters[encounters['PAYER_COVERAGE'] <= upper_limit]

# Get the unique combinations of DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST for outliers and non-outliers
outlier_combinations = set(zip(outliers_above_upper['DESCRIPTION'], 
                                outliers_above_upper['REASONDESCRIPTION'], 
                                outliers_above_upper['BASE_ENCOUNTER_COST'],
                                outliers_above_upper['PAYER']))
non_outlier_combinations = set(zip(non_outliers['DESCRIPTION'], 
                                   non_outliers['REASONDESCRIPTION'], 
                                   non_outliers['BASE_ENCOUNTER_COST'],
                                   non_outliers['PAYER']))

# Find common combinations between the outliers and non-outliers based on DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST
common_combinations = outlier_combinations.intersection(non_outlier_combinations)

# Create a DataFrame for common combinations
if common_combinations:
    common_combinations_df = pd.DataFrame(list(common_combinations), 
                                          columns=['DESCRIPTION', 'REASONDESCRIPTION', 'BASE_ENCOUNTER_COST','PAYER'])
    print("Common combinations between outliers and non-outliers:")
    print(common_combinations_df.to_string(index=False))  # Print the table without index
else:
    print("No common combinations between outliers and non-outliers.")

# Convert common combinations to a list of tuples
common_combinations_list = list(common_combinations)

# Filter the entire dataset (outliers and non-outliers) to check if the combination matches
matching_rows_whole_data = encounters[encounters.apply(
    lambda row: (row['DESCRIPTION'], row['REASONDESCRIPTION'], row['BASE_ENCOUNTER_COST'],row['PAYER']) in common_combinations_list, axis=1)]


Common combinations between outliers and non-outliers:
                                                           DESCRIPTION                           REASONDESCRIPTION  BASE_ENCOUNTER_COST                                PAYER
                                    Encounter for check up (procedure)                                        None                85.55 b3221cfc-24fb-339e-823d-bc4136cbc4ed
                                                        Prenatal visit                            Normal pregnancy               142.58 7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a
                                            Consultation for treatment                                        None               142.58 6e2f1a2d-27bd-3701-8d08-dae202c58632
                                  Emergency room admission (procedure)                                        None               146.18 6e2f1a2d-27bd-3701-8d08-dae202c58632
Periodic reevaluation and management of healthy individual (procedure)          

In [ ]:
# Calculate the total number of matching rows
total_matching_rows = len(matching_rows_whole_data)

# Calculate the percentage of matching rows in the entire dataset
percentage_matching_whole_data = (total_matching_rows / len(encounters)) * 100
# Print the results
print(f"Total number of rows with common DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST: {total_matching_rows} out of {encounters.shape[0]} ")
print(f"Percentage of rows in the entire dataset with common DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST: {percentage_matching_whole_data:.2f}%")

Total number of rows with common DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST: 8665 out of 27891 
Percentage of rows in the entire dataset with common DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST: 31.07%


Identifies rows where PAYER_COVERAGE is above a specified upper limit (outliers_above_upper) and below or equal to the limit (non_outliers).

Extracts unique combinations of DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST for both outliers and non-outliers.

Finds common combinations between outliers and non-outliers.

Filters the entire dataset to identify rows matching the common combinations.

Calculates and reports the total number and percentage of rows in the dataset with these common combinations.

In [ ]:
upper_limit = 389.4250

# Filter out the rows where PAYER_COVERAGE is above the upper limit and below or equal to the upper limit
outliers_above_upper = encounters[encounters['PAYER_COVERAGE'] > upper_limit]
non_outliers = encounters[encounters['PAYER_COVERAGE'] <= upper_limit]

# Get the unique combinations of DESCRIPTION, REASONDESCRIPTION, BASE_ENCOUNTER_COST, and PAYER for outliers and non-outliers
outlier_combinations = set(zip(outliers_above_upper['DESCRIPTION'], 
                                outliers_above_upper['REASONDESCRIPTION'], 
                                outliers_above_upper['BASE_ENCOUNTER_COST'],
                                outliers_above_upper['PAYER']))
non_outlier_combinations = set(zip(non_outliers['DESCRIPTION'], 
                                   non_outliers['REASONDESCRIPTION'], 
                                   non_outliers['BASE_ENCOUNTER_COST'],
                                   non_outliers['PAYER']))

# Find common combinations between the outliers and non-outliers
common_combinations = outlier_combinations.intersection(non_outlier_combinations)

# Convert common combinations to a list of tuples
common_combinations_list = list(common_combinations)

# Filter the entire dataset to retrieve rows with matching combinations
matching_rows = encounters[encounters.apply(
    lambda row: (row['DESCRIPTION'], row['REASONDESCRIPTION'], row['BASE_ENCOUNTER_COST'], row['PAYER']) in common_combinations_list, axis=1)]

# Sort the matching rows by DESCRIPTION, REASONDESCRIPTION, BASE_ENCOUNTER_COST, and PAYER_COVERAGE
matching_rows_sorted_PAYER_COVERAGE = matching_rows.sort_values(by=['DESCRIPTION', 'REASONDESCRIPTION', 'BASE_ENCOUNTER_COST', 'PAYER_COVERAGE'])

# Display the organized matching rows
print(f"Total number of matching rows: {len(matching_rows_sorted_PAYER_COVERAGE)}")
print(matching_rows_sorted_PAYER_COVERAGE)


Total number of matching rows: 8665
                                         Id                 START  \
6010   2f1ad566-7ad4-5941-c746-1978da702a39  2014-01-13T16:18:52Z   
27869  b47f7f7b-05fa-e86d-01cf-8fa3955d2817  2022-01-25T23:36:18Z   
22427  96b3b2bf-94f2-2336-591e-e0ef0d4527df  2020-05-15T09:02:59Z   
11636  c4088f6f-80b0-8af7-ea72-0ea3f56b13fa  2015-10-01T18:53:48Z   
3670   42b7c94b-71a2-1b0e-c132-646f52e35fa4  2013-02-12T03:21:34Z   
...                                     ...                   ...   
10881  36d728a0-0bb8-85e0-ca4f-68af4e5658cd  2015-06-05T15:31:19Z   
11576  eebd3962-97f1-dfd6-19aa-a5413ea60340  2015-09-21T09:05:31Z   
21313  3aa19457-1b0d-f89c-c67f-e5895ef6338c  2019-11-08T23:09:43Z   
26320  4bbd6f36-c074-d545-b074-ae83a0d625de  2021-07-13T15:23:00Z   
4676   99ca2682-b448-e164-d868-f5030f59c2ad  2013-06-24T07:09:49Z   

                       STOP                               PATIENT  \
6010   2014-01-14T16:18:52Z  6b495db9-938d-ef65-ee0d-1d4224e77157 

In [ ]:
upper_limit = 389.4250

# Filter out the rows where TOTAL_CLAIM_COST is above the upper limit and below the upper limit
outliers_above_upper = encounters[encounters['PAYER_COVERAGE'] > upper_limit]

# Get the unique combinations of DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST for the outliers
outlier_combinations = set(zip(outliers_above_upper['DESCRIPTION'], 
                                outliers_above_upper['REASONDESCRIPTION'], 
                                outliers_above_upper['BASE_ENCOUNTER_COST'],
                                outliers_above_upper['PAYER']))

# Create a DataFrame for the outlier combinations
if outlier_combinations:
    outlier_combinations_df = pd.DataFrame(list(outlier_combinations), 
                                           columns=['DESCRIPTION', 'REASONDESCRIPTION', 'BASE_ENCOUNTER_COST','PAYER'])
    print("Combinations for rows where TOTAL_CLAIM_COST is above the upper limit:")
    print(outlier_combinations_df.to_string(index=False))  # Print the table without index
else:
    print("No combinations for rows where TOTAL_CLAIM_COST is above the upper limit.")


Combinations for rows where TOTAL_CLAIM_COST is above the upper limit:
                                                           DESCRIPTION                                        REASONDESCRIPTION  BASE_ENCOUNTER_COST                                PAYER
                                                        Prenatal visit                          Fetus with unknown complication               142.58 6e2f1a2d-27bd-3701-8d08-dae202c58632
Periodic reevaluation and management of healthy individual (procedure)                                                     None               146.18 7caa7254-5050-3b5e-9eae-bd5ea30e809c
                                              Encounter for 'check-up'                                                     None                85.55 b3221cfc-24fb-339e-823d-bc4136cbc4ed
                            General examination of patient (procedure)                                                     None               136.80 7caa7254-5050-3b5e-9eae-bd5ea30e809c

In [ ]:
# Calculate the total number of outlier rows
total_outlier_rows = len(outliers_above_upper)

# Calculate the percentage of outlier rows in the entire dataset
percentage_outliers = (total_outlier_rows / len(encounters)) * 100
# Print the results
print(f"Total number of rows with TOTAL_CLAIM_COST above the upper limit: {total_outlier_rows} out of {encounters.shape[0]}")
print(f"Percentage of rows in the entire dataset with TOTAL_CLAIM_COST above the upper limit: {percentage_outliers:.2f}%")

Total number of rows with TOTAL_CLAIM_COST above the upper limit: 5035 out of 27891
Percentage of rows in the entire dataset with TOTAL_CLAIM_COST above the upper limit: 18.05%


Identifies rows where PAYER_COVERAGE exceeds a specified upper limit (outliers_above_upper).

Extracts unique combinations of DESCRIPTION, REASONDESCRIPTION, and BASE_ENCOUNTER_COST for these outliers.

Reports the total number and percentage of outlier rows in the dataset is equal to the IQR outliers and percentage.

##### Summary:

Through the analysis of rows based on `PAYER_COVERAGE`, we explored outliers exceeding a specified upper limit and their relationship with the rest of the dataset. This exploration involved:

1. **Comparison of Outliers and Non-Outliers**:  
   - Unique combinations of `DESCRIPTION`, `REASONDESCRIPTION`, and `BASE_ENCOUNTER_COST` were identified for both outliers and non-outliers.  
   - Common patterns between these groups were determined to understand shared characteristics and their distribution across the dataset.  

2. **Outlier-Specific Analysis**:  
   - Unique combinations for outliers were analyzed independently to evaluate their distinct traits.  
   - The total number and percentage of outlier rows were calculated to assess the impact of extreme values in the dataset.  

3. **Dataset-Wide Matching**:  
   - The dataset was filtered to identify rows that match the common combinations between outliers and non-outliers.  
   - The prevalence of these common patterns was measured as a percentage of the entire dataset.  

##### Best Action:  
Given the findings, **removing the outliers is the recommended approach** to maintain the quality of the dataset. This ensures that extreme values in `PAYER_COVERAGE` do not skew subsequent analyses or lead to misleading conclusions.

#### patients (LAT) (LON)

In [ ]:
# Check for valid geographical locations range for LAT and LON
# LAT should be between  0 and 90 degrees.
# LON should be between -180 and 180 degrees.
print("LAT Range:", patients['LAT'].min(), "to", patients['LAT'].max())
print("LON Range:", patients['LON'].min(), "to", patients['LON'].max())


LAT Range: 42.20492119340221 to 42.49546445415797
LON Range: -71.16564805 to -70.73082425


In [ ]:
# Create a dictionary with the required values
summary = {
    "Total rows": patients.shape[0],
    "Unique Ids": patients['Id'].nunique(),
    "Unique LAST": patients['LAST'].nunique(),
    "Unique LAT": patients['LAT'].nunique(),
    "Unique LON": patients['LON'].nunique()
}

# Print the summary
for key, value in summary.items():
    print(f"{key}: {value}")

Total rows: 973
Unique Ids: 973
Unique LAST: 498
Unique LAT: 973
Unique LON: 973


In [ ]:
# Step 1: Filter for Mr. and MARITAL = M
mr_rows = patients[(patients['PREFIX'] == 'Mr.') & (patients['MARITAL'] == 'M')]

# Step 2: Filter for Mrs. and MARITAL = M
mrs_rows = patients[(patients['PREFIX'] == 'Mrs.')]

# Step 3: Initialize an empty list to store the final results
final_results = []

# Step 4: For each Mr. row, find the corresponding Mrs. row where LAST == MAIDEN
for _, mr_row in mr_rows.iterrows():
    matching_mrs_rows = mrs_rows[mrs_rows['MAIDEN'] == mr_row['LAST']]
    if not matching_mrs_rows.empty:
        # Append Mr. row followed by the matching Mrs. rows
        final_results.append(mr_row.to_dict())
        final_results.extend(matching_mrs_rows.to_dict(orient='records'))

# Step 5: Convert the final results into a DataFrame
final_df = pd.DataFrame(final_results)

print(final_df)

                                       Id   BIRTHDATE   DEATHDATE PREFIX  \
0    8e9650d1-788a-78f9-4a28-d08f7f95354a  1928-12-25                Mr.   
1    334ef90d-932c-005d-9714-543546cc7a93  1982-10-19               Mrs.   
2    f4cde0dc-db22-5d98-5228-55121ae74cad  1946-01-09  2012-09-29   Mrs.   
3    229ce85d-1487-38af-738f-2efb401334c0  1933-10-28               Mrs.   
4    720560d4-51da-c38c-ee90-c15935278df1  1972-06-27                Mr.   
..                                    ...         ...         ...    ...   
403  3854c2ea-24a8-1aaa-9d17-3a5c09a10a58  1980-11-12               Mrs.   
404  9107aa45-d481-a887-df1f-b1e6f66c8181  1939-07-05  2021-03-08   Mrs.   
405  a950f10e-c0e0-5b7f-14d7-5e750044ccec  1936-06-26               Mrs.   
406  c06513da-7f35-b4eb-5bab-28c87ff97a10  1959-03-04  2014-11-16    Mr.   
407  4d55540d-c43d-e6e8-5364-ec2509df31e3  1964-10-15               Mrs.   

            FIRST            LAST SUFFIX          MAIDEN MARITAL   RACE  \
0      Frank

In [ ]:
# Step 1: Filter for Mr. and MARITAL = M
mr_rows = patients[(patients['PREFIX'] == 'Mr.') & (patients['MARITAL'] == 'M')]

# Step 2: Filter for Mrs. rows (no MARITAL filter for Mrs.)
mrs_rows = patients[(patients['PREFIX'] == 'Mrs.')]

# Step 3: Initialize an empty list to store the final results
final_results = []

# Step 4: For each Mr. row, find the corresponding Mrs. row where LAST == MAIDEN and additional matches
for _, mr_row in mr_rows.iterrows():
    # Filter Mrs. rows where MAIDEN matches LAST
    matching_mrs_rows = mrs_rows[mrs_rows['MAIDEN'] == mr_row['LAST']]
    
    # Further filter to ensure that ADDRESS, CITY, COUNTY, and ZIP match
    matching_mrs_rows = matching_mrs_rows[
        (matching_mrs_rows['ADDRESS'] == mr_row['ADDRESS']) &
        (matching_mrs_rows['CITY'] == mr_row['CITY']) &
        (matching_mrs_rows['COUNTY'] == mr_row['COUNTY']) &
        (matching_mrs_rows['ZIP'] == mr_row['ZIP'])
    ]
    
    if not matching_mrs_rows.empty:
        # Append Mr. row followed by the matching Mrs. rows
        final_results.append(mr_row.to_dict())
        final_results.extend(matching_mrs_rows.to_dict(orient='records'))

# Step 5: Convert the final results into a DataFrame
final_df2 = pd.DataFrame(final_results)

# Step 6: Print the final DataFrame
print(final_df2)


Empty DataFrame
Columns: []
Index: []


In [ ]:
print("Unique ADDRESS:", patients['ADDRESS'].nunique())

Unique ADDRESS: 973


##### Summary:

This analysis explores the geographical and demographic details of patients, with a focus on validating unique attributes, geographical ranges, and relationships within the dataset.

1. **Geographical Validation**:
   - **Latitude (LAT)**: Verified to fall within the valid range of 0 to 90 degrees.  
   - **Longitude (LON)**: Verified to fall within the valid range of -180 to 180 degrees.  
   - The unique counts for `LAT` and `LON` are consistent with the total number of rows (973), confirming that each row has a unique location.  

2. **Uniqueness Check**:
   - **Unique IDs**: Confirmed to match the total number of rows (973), ensuring each patient has a unique identifier.  
   - **Unique LAST Names**: Identified 498 unique values, indicating potential shared surnames among patients.  
   - **Unique LAT and LON**: Confirmed 973 unique values each, demonstrating no overlapping locations.  

3. **Marital and Relationship Analysis**:
   - Attempted to match records of individuals with the prefix `Mr.` and marital status `M` to those with the prefix `Mrs.` based on `LAST` and `MAIDEN` names.  
   - A refined approach further filtered matches by additional fields (`ADDRESS`, `CITY`, `COUNTY`, and `ZIP`), but no valid matches were found.  
   - This null result suggests that either such relationships are absent in the dataset or additional data attributes are needed for validation.  

4. **Address Uniqueness**:
   - The `ADDRESS` column was confirmed to have 973 unique values, equal to the total number of rows, indicating no shared addresses among patients.  
   - This supports the conclusion that `LAT` and `LON` values are valid and not outliers, justifying their retention in the dataset.
   - However, after realizing that there are 973 unique address values, I stopped further analysis. This indicates that they live in different places, and the assumption that they might be family members living together was unnecessary.

##### Best Action:
Given the above findings, **retain all rows and attributes** in the dataset. The geographical details (`LAT`, `LON`) and demographic attributes (`ID`, `LAST`, `ADDRESS`) are verified to be accurate and do not require adjustments or removal.

#### procedures (BASE_COST)

In [ ]:
upper_limit = 1768.5

# Filter out the rows where BASE_COST is above the upper limit and below the upper limit
outliers_above_upper = procedures[procedures['BASE_COST'] > upper_limit]
non_outliers = procedures[procedures['BASE_COST'] <= upper_limit]

# Get the unique combinations of DESCRIPTION and REASONDESCRIPTION for outliers and non-outliers
outlier_descriptions_reasons = set(zip(outliers_above_upper['DESCRIPTION'], outliers_above_upper['REASONDESCRIPTION']))
non_outlier_descriptions_reasons = set(zip(non_outliers['DESCRIPTION'], non_outliers['REASONDESCRIPTION']))

# Find common combinations between the outliers and non-outliers based on both DESCRIPTION and REASONDESCRIPTION
common_combinations = outlier_descriptions_reasons.intersection(non_outlier_descriptions_reasons)

# Create a DataFrame for common combinations
if common_combinations:
    common_combinations_df = pd.DataFrame(list(common_combinations), columns=['DESCRIPTION', 'REASONDESCRIPTION'])
    print("Common combinations between outliers and non-outliers:")
    print(common_combinations_df.to_string(index=False))  # Print the table without index
else:
    print("No common descriptions and reason descriptions between outliers and non-outliers.")

# Convert common combinations to a list of tuples
common_combinations_list = list(common_combinations)

# Filter the entire dataset (outliers and non-outliers) to check if the combination matches
matching_rows_whole_data = procedures[procedures.apply(
    lambda row: (row['DESCRIPTION'], row['REASONDESCRIPTION']) in common_combinations_list, axis=1)]


Common combinations between outliers and non-outliers:
                                                                                              DESCRIPTION                           REASONDESCRIPTION
                                                                                       Urine protein test                            Normal pregnancy
                                                                                Hepatitis C antibody test                            Normal pregnancy
                                                                                       Biopsy of prostate                                        None
                                                                               Throat culture (procedure)        Streptococcal sore throat (disorder)
                                                           Measurement of Varicella-zoster virus antibody                            Normal pregnancy
                                             

In [ ]:
# Calculate the total number of matching rows
total_matching_rows = len(matching_rows_whole_data)

# Calculate the percentage of matching rows in the entire dataset
percentage_matching_whole_data = (total_matching_rows / len(procedures)) * 100
# Print the results
print(f"Total number of rows with common DESCRIPTION and REASONDESCRIPTION: {total_matching_rows} out of {procedures.shape[0]}")
print(f"Percentage of rows in the entire dataset with common DESCRIPTION and REASONDESCRIPTION: {percentage_matching_whole_data:.2f}%")

Total number of rows with common DESCRIPTION and REASONDESCRIPTION: 3698 out of 47701
Percentage of rows in the entire dataset with common DESCRIPTION and REASONDESCRIPTION: 7.75%


Identifies rows where BASE_COST exceeds a specified upper limit (outliers_above_upper) and below or equal to the limit (non_outliers).

Extracts unique combinations of DESCRIPTION and REASONDESCRIPTION for both outliers and non-outliers.

Finds common combinations between outliers and non-outliers based on DESCRIPTION and REASONDESCRIPTION.

Filters the entire dataset to identify rows matching the common combinations.

Calculates and reports the total number and percentage of rows in the dataset with these common combinations.

In [ ]:
upper_limit = 1768.5

# Filter out the rows where BASE_COST is above the upper limit and below or equal to the upper limit
outliers_above_upper = procedures[procedures['BASE_COST'] > upper_limit]
non_outliers = procedures[procedures['BASE_COST'] <= upper_limit]

# Get the unique combinations of DESCRIPTION and REASONDESCRIPTION for outliers and non-outliers
outlier_descriptions_reasons = set(zip(outliers_above_upper['DESCRIPTION'], outliers_above_upper['REASONDESCRIPTION']))
non_outlier_descriptions_reasons = set(zip(non_outliers['DESCRIPTION'], non_outliers['REASONDESCRIPTION']))

# Find common combinations between the outliers and non-outliers
common_combinations = outlier_descriptions_reasons.intersection(non_outlier_descriptions_reasons)

# Convert common combinations to a list of tuples
common_combinations_list = list(common_combinations)

# Filter the `procedures` dataset to retrieve rows with matching combinations
matching_rows = procedures[procedures.apply(
    lambda row: (row['DESCRIPTION'], row['REASONDESCRIPTION']) in common_combinations_list, axis=1)]

# Sort the matching rows by DESCRIPTION and REASONDESCRIPTION
matching_rows_sorted_procedures = matching_rows.sort_values(by=['DESCRIPTION', 'REASONDESCRIPTION'])

# Display the organized matching rows
print(f"Total number of matching rows: {len(matching_rows_sorted_procedures)}")
print(matching_rows_sorted_procedures)


Total number of matching rows: 3698
                      START                  STOP  \
92     2011-02-10T15:41:08Z  2011-02-10T15:56:08Z   
1759   2012-03-05T17:55:09Z  2012-03-05T18:10:09Z   
2004   2012-03-22T16:11:43Z  2012-03-22T16:26:43Z   
2148   2012-04-04T02:02:47Z  2012-04-04T02:17:47Z   
2179   2012-04-07T14:56:58Z  2012-04-07T15:11:58Z   
...                     ...                   ...   
45979  2021-08-21T13:00:16Z  2021-08-21T13:15:16Z   
46817  2021-11-02T21:01:04Z  2021-11-02T21:16:04Z   
47152  2021-12-04T20:35:37Z  2021-12-04T20:50:37Z   
47524  2022-01-08T21:16:09Z  2022-01-08T21:31:09Z   
47652  2022-01-21T21:34:23Z  2022-01-21T21:49:23Z   

                                    PATIENT  \
92     8fe3348d-477f-541e-96df-bbe5b552104f   
1759   9eb99e6f-21e9-c2dc-835f-13c6a53f9a16   
2004   bc4bc1c7-5887-80a6-383b-16dd61054474   
2148   394f3e43-784c-b938-adb6-4b69efd46bbf   
2179   fea2a30d-f7c9-eeb0-9aad-5c711f55574f   
...                                     ...  

In [ ]:
upper_limit = 1768.5

# Filter out the rows where BASE_COST is above the upper limit (outliers)
outliers_above_upper = procedures[procedures['BASE_COST'] > upper_limit]

# Get the unique combinations of DESCRIPTION and REASONDESCRIPTION for the outliers
outlier_descriptions_reasons = set(zip(outliers_above_upper['DESCRIPTION'], outliers_above_upper['REASONDESCRIPTION']))

# Create a DataFrame for the outlier combinations
if outlier_descriptions_reasons:
    outlier_combinations_df = pd.DataFrame(list(outlier_descriptions_reasons), columns=['DESCRIPTION', 'REASONDESCRIPTION'])
    print("Combinations for rows where BASE_COST is above the upper limit (outliers):")
    print(outlier_combinations_df.to_string(index=False))  # Print the table without index
else:
    print("No combinations for rows where BASE_COST is above the upper limit.")


Combinations for rows where BASE_COST is above the upper limit (outliers):
                                                                                              DESCRIPTION                                                     REASONDESCRIPTION
                                                  Combined chemotherapy and radiation therapy (procedure)                               Overlapping malignant neoplasm of colon
                                                                               Subcutaneous immunotherapy                                                                  None
                                                                               Throat culture (procedure)                                    Acute viral pharyngitis (disorder)
                                                                            Bone density scan (procedure)                                                                  None
                                             

In [ ]:
# Calculate the total number of matching outlier rows
total_outlier_matching_rows = len(outliers_above_upper)

# Calculate the percentage of matching outlier rows in the outliers dataset
percentage_outlier_matching_rows = (total_outlier_matching_rows / len(procedures)) * 100

# Print the results
print(f"Total number of matching outlier rows: {total_outlier_matching_rows} out of {procedures.shape[0]}")
print(f"Percentage of matching outlier rows in the outliers dataset: {percentage_outlier_matching_rows:.2f}%")

Total number of matching outlier rows: 9174 out of 47701
Percentage of matching outlier rows in the outliers dataset: 19.23%


Identifies rows where BASE_COST exceeds the specified upper limit (outliers_above_upper).

Extracts unique combinations of DESCRIPTION and REASONDESCRIPTION for these outliers.

Reports the total number and percentage of outlier rows in the dataset is equal to the IQR outliers and percentage.

##### Summary:

In this analysis, we focused on identifying and examining rows where `BASE_COST` exceeds a specified upper limit. The process involved:

1. **Comparison of Outliers and Non-Outliers**:  
   - Unique combinations of `DESCRIPTION` and `REASONDESCRIPTION` were identified for both outliers and non-outliers.  
   - Common patterns between these two groups were examined to detect any shared traits across different subsets of the data.

2. **Outlier-Specific Analysis**:  
   - The analysis further delved into the unique combinations for outliers, specifically focusing on `BASE_COST` values that exceeded the upper limit.  
   - The total number and percentage of outlier rows in the dataset were calculated to understand the extent of the impact of these outliers.

3. **Dataset-Wide Matching**:  
   - Rows matching the common combinations between outliers and non-outliers were filtered from the entire dataset.  
   - The percentage of rows in the entire dataset that matched these common combinations was reported.

##### Best Action:  
Considering the impact of extreme `BASE_COST` values, **removing the outliers is recommended** to improve the quality and reliability of the dataset. This step will prevent skewed results and ensure more accurate analysis moving forward.

#### Daily_Town_COVID19 (covid_cases_cumulative)  (daily_cases)

We need to get the **maximum value** of `covid_cases_cumulative` for each town within a county and then **sum** them with the `sum_daily_cases` for each county, you can follow these steps:

1. **Group by County and Town**: First, calculate the **maximum** of `covid_cases_cumulative` for each town.
2. **Sum the Daily Cases**: Then, calculate the **sum** of `daily_cases` for each town.
3. **Aggregate by County**: Finally, for each county, sum the maximum values of `covid_cases_cumulative` and the sum of `daily_cases` from all the towns within that county.


Explanation:

1. **Step 1: Group by `COUNTY`**:
   - **`total_max_covid_cases_cumulative`**: For each county, we sum up the maximum values of `covid_cases_cumulative` from all towns in that county.
   - **`total_sum_daily_cases`**: For each county, we sum up the daily cases from all towns in that county.

2. **Step 2: Group by `Town`**:
   - **`max_covid_cases_cumulative`**: We calculate the maximum value of `covid_cases_cumulative` for each town.
   - **`sum_daily_cases`**: We calculate the sum of `daily_cases` for each town.




In [ ]:
# Step 1: Group by County and Town, calculate the max of covid_cases_cumulative and sum of daily_cases for each town
town_summary = Daily_Town_COVID19.groupby(['COUNTY', 'Town']) \
    .agg(
        max_covid_cases_cumulative=('covid_cases_cumulative', 'max'),
        sum_daily_cases=('daily_cases', 'sum')
    ).reset_index()

# Step 2: Group by County and aggregate the results
county_summary = town_summary.groupby('COUNTY') \
    .agg(
        total_max_covid_cases_cumulative=('max_covid_cases_cumulative', 'sum'),
        total_sum_daily_cases=('sum_daily_cases', 'sum')
    ).reset_index()

print(county_summary)


      COUNTY  total_max_covid_cases_cumulative  total_sum_daily_cases
0      Essex                             71834                  71834
1  Middlesex                             93862                  93862
2    Norfolk                             35739                  35739
3   Plymouth                             32245                  32245
4    Suffolk                             66066                  66066


Now lets group the `Daily_Town_COVID19` dataset by each town and calculates two metrics for each: the maximum cumulative COVID-19 cases (`max_covid_cases_cumulative`) and the total sum of daily cases (`sum_daily_cases`). The results are compiled into a summary table where each row represents a town, and the index is reset for a flat structure.

In [ ]:
town_summary = Daily_Town_COVID19.groupby('Town') \
    .agg(
        max_covid_cases_cumulative=('covid_cases_cumulative', 'max'),
        sum_daily_cases=('daily_cases', 'sum')
    ).reset_index()

print(town_summary)


           Town  max_covid_cases_cumulative  sum_daily_cases
0      Abington                        1139             1139
1         Acton                         633              633
2      Amesbury                         808              808
3       Andover                        1838             1838
4     Arlington                        1264             1264
..          ...                         ...              ...
142  Wilmington                        1603             1603
143  Winchester                         876              876
144    Winthrop                        1736             1736
145      Woburn                        3144             3144
146    Wrentham                         722              722

[147 rows x 3 columns]


In [ ]:
# Check if the values in 'max_covid_cases_cumulative' and 'sum_daily_cases' are equal for each town
town_summary['columns_are_equal'] = town_summary['max_covid_cases_cumulative'] == town_summary['sum_daily_cases']

# Count matching and non-matching rows
matching_rows = town_summary['columns_are_equal'].sum()
non_matching_rows = len(town_summary) - matching_rows

# Print the results
print(f"Number of matching rows: {matching_rows}")
print(f"Number of non-matching rows: {non_matching_rows}")

Number of matching rows: 147
Number of non-matching rows: 0


Revisit Thresholds:
   - Using thresholds specifically calculated for the aggregated data.
   - Recalculate IQR and Z-scores at the town level.


In [ ]:
# Function to detect outliers using IQR
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] < lower_bound) | (data[column] > upper_bound)]

# Function to detect outliers using z-score
def detect_outliers_zscore(data, column):
    threshold = 3  # Define the threshold for Z-score outliers (e.g., 3 standard deviations)
    mean = data[column].mean()
    std = data[column].std()
    z_scores = (data[column] - mean) / std
    return data[(z_scores > threshold) | (z_scores < -threshold)]

# Apply outlier detection to all datasets and compile results into a table
def analyze_outliers(data_dict):
    outlier_results = []

    for dataset_name, data in data_dict.items():
        for column in data.select_dtypes(include=[np.number]).columns:
            
            # IQR Method
            iqr_outliers = detect_outliers_iqr(data, column)
            iqr_count = len(iqr_outliers)
            iqr_percentage = (iqr_count / data.shape[0]) * 100  # Calculate percentage of IQR outliers
            
            # Z-score Method
            zscore_outliers = detect_outliers_zscore(data, column)
            zscore_count = len(zscore_outliers)
            
            # Append results to the table
            outlier_results.append({
                'Dataset': dataset_name,
                'Column': column,
                'IQR Outliers': iqr_count,
                'IQR Percentage': f"{iqr_percentage:.2f}%",
                'Z-Score Outliers': zscore_count,
            })
    
    # Convert the results to a DataFrame for easy display
    outlier_df = pd.DataFrame(outlier_results)
    return outlier_df

# Example: Analyze outliers for all datasets
data_dict = {
    'town_summary': town_summary,

}

outlier_df = analyze_outliers(data_dict)
outlier_df


,Dataset,Column,IQR Outliers,IQR Percentage,Z-Score Outliers
0,town_summary,max_covid_cases_cumulative,14,9.52%,2
1,town_summary,sum_daily_cases,14,9.52%,2


Analyze Results:
The analysis using both IQR and Z-score methods revealed a small proportion of outliers IQR Percentage (9.52%) and Z-Score Outliers (2) in the columns max_covid_cases_cumulative and sum_daily_cases. Since these outliers represent valid extreme values and their proportion is not large enough to significantly affect the overall analysis, they will be retained in the dataset

### Dealing With The Outliers

Based on the previous exploration of the outliers, we will be dealing with the following:

- encounters (TOTAL_CLAIM_COST)
- encounters (PAYER_COVERAGE)
- procedures (BASE_COST)

#### encounters (TOTAL_CLAIM_COST)

In [ ]:
encounters.shape[0]

27891

In [ ]:
# Step 1: Filter based on upper limit
upper_limit = 3317.4550
encounters = encounters[encounters['TOTAL_CLAIM_COST'] <= upper_limit]

# Step 2: Remove based on IQR
Q1 = encounters['TOTAL_CLAIM_COST'].quantile(0.25)
Q3 = encounters['TOTAL_CLAIM_COST'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
encounters = encounters[(encounters['TOTAL_CLAIM_COST'] >= lower_bound) & 
                        (encounters['TOTAL_CLAIM_COST'] <= upper_bound)]

# Step 3: Remove based on Z-score
z_scores = np.abs(stats.zscore(encounters['TOTAL_CLAIM_COST']))
encounters = encounters[z_scores < 3]

# Check the size after cleaning
print(f"Cleaned encounters data size for TOTAL_CLAIM_COST: {encounters.shape[0]}")

Cleaned encounters data size for TOTAL_CLAIM_COST: 21301


In [ ]:
TOTAL_CLAIM_COST_IQR = {
    'encounters': encounters[['TOTAL_CLAIM_COST']]
}

TOTAL_CLAIM_COST_IQR = analyze_outliers(TOTAL_CLAIM_COST_IQR)
TOTAL_CLAIM_COST_IQR


,Dataset,Column,IQR Outliers,IQR Percentage,Z-Score Outliers
0,encounters,TOTAL_CLAIM_COST,1287,6.04%,0


#### encounters (PAYER_COVERAGE)

In [ ]:
# Step 1: Filter based on upper limit
upper_limit = 389.4250
encounters = encounters[encounters['PAYER_COVERAGE'] <= upper_limit]

# Step 2: Remove based on IQR
Q1 = encounters['PAYER_COVERAGE'].quantile(0.25)
Q3 = encounters['PAYER_COVERAGE'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
encounters = encounters[(encounters['PAYER_COVERAGE'] >= lower_bound) & 
                        (encounters['PAYER_COVERAGE'] <= upper_bound)]

# Step 3: Remove based on Z-score
z_scores = np.abs(stats.zscore(encounters['PAYER_COVERAGE']))
encounters = encounters[z_scores < 3]

# Check the size after cleaning
print(f"Cleaned encounters data size for PAYER_COVERAGE: {encounters.shape[0]}")

Cleaned encounters data size for PAYER_COVERAGE: 18183


- Initial Data Size (27,891): This is the dataset before any outlier detection and removal.
- After Filtering (the value of `TOTAL_CLAIM_COST` exceeds a specified threshold) (21,301): The data is reduced after filtering IQR. This is expected because extreme values that do not fit within the set threshold are removed.
- After Filtering (the value of `PAYER_COVERAGE` exceeds a specified threshold)(18,183): Further filtering based also on the IQR method includes values that are exceeding the upper limit. These values are also considered outliers, so removing them further reduces the dataset size.

#### procedures (BASE_COST)

In [ ]:
procedures.shape[0]

47701

In [ ]:
procedures['BASE_COST'].value_counts()


BASE_COST
431      30988
31          45
30          43
32          34
29          28
         ...  
38182        1
8346         1
9824         1
6379         1
10446        1
Name: count, Length: 8268, dtype: int64

In [ ]:
# Step 1: Filter based on upper limit
upper_limit = 1768.5
procedures = procedures[procedures['BASE_COST'] <= upper_limit]

# Step 2: Temporarily exclude the dominant value (e.g., 431) for IQR calculation
dominant_value = 431
filtered = procedures[procedures['BASE_COST'] != dominant_value]

# Step 3: Compute IQR without the dominant value
Q1 = filtered['BASE_COST'].quantile(0.25)
Q3 = filtered['BASE_COST'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Step 4: Apply IQR filtering back to the original data
procedures = procedures[
    (procedures['BASE_COST'] >= lower_bound) & 
    (procedures['BASE_COST'] <= upper_bound)
]

# Step 5: Check the size and distribution after cleaning
print(f"Cleaned procedures data size for BASE_COST: {procedures.shape[0]}")


Cleaned procedures data size for BASE_COST: 38527


In [ ]:
procedures['BASE_COST'].value_counts()


BASE_COST
431     30988
31         45
30         43
32         34
29         28
        ...  
281         1
1362        1
1476        1
1493        1
1096        1
Name: count, Length: 1627, dtype: int64

In [ ]:
BASE_COST_IQR = {
    'encounters': procedures[['BASE_COST']]
}

BASE_COST_IQR = analyze_outliers(BASE_COST_IQR)
BASE_COST_IQR


,Dataset,Column,IQR Outliers,IQR Percentage,Z-Score Outliers
0,encounters,BASE_COST,7539,19.57%,1450


##### **Overview**
The `BASE_COST` column in the `procedures` dataset required careful preprocessing due to its skewed distribution, dominated by a single value (`431`). After evaluating various filtering techniques, it was determined that retaining only rows with `BASE_COST == 431` provided the most meaningful dataset for further analysis.

---

##### **Steps Taken**
1. **Initial Exploration**:
   - The dataset contained **47,701 rows** with `BASE_COST` values ranging from **1 to 289,531**.
   - A significant majority of rows (30,988, or ~65%) had a `BASE_COST` value of **431**.

2. **Filtering Approaches Evaluated**:
   - **Upper Limit Filter**:
     - Initially set an upper limit of `1768.5` to remove outliers.
     - However, this approach failed to meaningfully reduce noise, as the remaining data still showed extreme sparsity across non-`431` values.
   - **IQR (Interquartile Range)**:
     - Used the IQR method to remove extreme outliers.
     - Despite being a standard approach, it did not resolve the dominance of `431` or the sparsity issue effectively.
   - **Z-Score**:
     - Evaluated this statistical method but found it unsuitable due to the heavy-tailed distribution of the data.

3. **Final Decision**:
   - Retained only rows where `BASE_COST == 431`, given its overwhelming presence and relevance.
   - This decision significantly simplified the dataset and ensured meaningful insights could be drawn.

---

##### **Outcome**
  - Rows remained: **30,988** (100% of `BASE_COST == 431`).
  - Removed all other sparse and inconsistent values to focus on the dominant pattern.

---

##### **Justification for Retaining Only `431`**
- The dominance of `431` made other values statistically insignificant.
- Including sparse values could lead to misleading insights or unnecessary complexity in further analyses.


In [ ]:
# Step 1: Retain only rows with BASE_COST == 431
procedures = procedures[procedures['BASE_COST'] == 431]

# Step 2: Check the size and distribution after filtering
print(f"Filtered procedures data size for BASE_COST: {procedures.shape[0]}")

Filtered procedures data size for BASE_COST: 30988


The dataset reduction from 47,701 to 30,988 rows is normal after applying this filtering technique. We used IQR filtering only, without applying Z-score filtering, because Z-score filtering removed all rows. The reason for this is that Z-score measures how far each data point is from the mean in terms of standard deviations. After applying IQR filtering, the dataset might already be tightly distributed around the mean but still exceed the Z-score threshold of 3. Each step in the outlier removal process reduces the dataset, and applying Z-score filtering as the final step might overly restrict the data.

### Final Check for Outliers

In [ ]:
# Apply outlier detection to all datasets and compile results into a table
def analyze_outliers(data_dict, columns_to_analyze):
    outlier_results = []

    for dataset_name, data in data_dict.items():
        # Get columns to analyze for the current dataset
        if dataset_name in columns_to_analyze:
            columns = columns_to_analyze[dataset_name]
        else:
            continue  # Skip if no columns are specified for this dataset

        for column in columns:
            if column not in data.columns:
                continue  # Skip if the column does not exist in the dataset

            # IQR Method
            iqr_outliers = detect_outliers_iqr(data, column)
            iqr_count = len(iqr_outliers)
            iqr_percentage = (iqr_count / data.shape[0]) * 100  # Calculate percentage of IQR outliers

            # Z-score Method
            zscore_outliers = detect_outliers_zscore(data, column)
            zscore_count = len(zscore_outliers)

            # Append results to the table
            outlier_results.append({
                'Dataset': dataset_name,
                'Column': column,
                'IQR Outliers': iqr_count,
                'IQR Percentage': f"{iqr_percentage:.2f}%",
                'Z-Score Outliers': zscore_count,
            })

    # Convert the results to a DataFrame for easy display
    outlier_df = pd.DataFrame(outlier_results)
    return outlier_df



data_dict = {
    'encounters': encounters,  
    'procedures': procedures   
}

columns_to_analyze = {
    'encounters': ['TOTAL_CLAIM_COST', 'PAYER_COVERAGE'],
    'procedures': ['BASE_COST']
}

outlier_df2 = analyze_outliers(data_dict, columns_to_analyze)
outlier_df2


,Dataset,Column,IQR Outliers,IQR Percentage,Z-Score Outliers
0,encounters,TOTAL_CLAIM_COST,3346,18.40%,425
1,encounters,PAYER_COVERAGE,0,0.00%,0
2,procedures,BASE_COST,0,0.00%,0


#### Conclusion: Why Not Drop the `TOTAL_CLAIM_COST` Outliers

The 3346 outliers in `TOTAL_CLAIM_COST` (18.40% of the data) appeared after dropping the outliers in `PAYER_COVERAGE` and after `TOTAL_CLAIM_COST` was already cleaned. These outliers reflect real-world variability in healthcare costs, particularly for uninsured patients (`PAYER_COVERAGE = 0`). They represent legitimate cases of high medical expenses when insurance is absent. Removing them would reduce the dataset and could introduce bias by losing valuable information about high-cost healthcare scenarios. Retaining these outliers ensures a more accurate and comprehensive view of healthcare costs, reflecting both insured and uninsured patient experiences.

# Trending Analysis

## Encounters: Trends in Encounter Costs

In [ ]:
# Convert 'START' to datetime
encounters['START'] = pd.to_datetime(encounters['START'])

# Group by year-month and calculate total costs
encounters['Year-Month'] = encounters['START'].dt.to_period('M')
cost_trends = encounters.groupby('Year-Month')['TOTAL_CLAIM_COST'].sum().reset_index()
cost_trends['Year-Month'] = cost_trends['Year-Month'].astype(str)

# Plot the trend
fig = px.line(cost_trends, x='Year-Month', y='TOTAL_CLAIM_COST',
              title='Total Encounter Costs Over Time',
              labels={'Year-Month': 'Year-Month', 'TOTAL_CLAIM_COST': 'Total Claim Cost'},
              markers=True)
fig.update_layout(xaxis=dict(title='Year-Month', tickangle=45), yaxis_title='Total Claim Cost ($)')
fig.show()


C:\Users\faisa\AppData\Local\Temp\ipykernel_15480\1831169698.py:5: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



## Patients: Age Group Distribution

In [ ]:
# Calculate age
current_year = pd.Timestamp.now().year
patients['BIRTHDATE'] = pd.to_datetime(patients['BIRTHDATE'])
patients['Age'] = current_year - patients['BIRTHDATE'].dt.year

# Create age groups
bins = [0, 18, 35, 50, 65, 80, 100]
labels = ['0-17', '18-34', '35-49', '50-64', '65-79', '80+']
patients['Age Group'] = pd.cut(patients['Age'], bins=bins, labels=labels, right=False)

# Group by age group
age_group_dist = patients['Age Group'].value_counts().reset_index()
age_group_dist.columns = ['Age Group', 'Count']

# Plot the distribution
fig = px.bar(age_group_dist, x='Age Group', y='Count',
             title='Patient Age Group Distribution',
             labels={'Count': 'Number of Patients'},
             text='Count')
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(xaxis_title='Age Group', yaxis_title='Number of Patients')
fig.show()


## Procedures: Monthly Procedure Volume

In [ ]:
procedures['START'] = pd.to_datetime(procedures['START'])

# Group by year-month and count procedures
procedures['Year-Month'] = procedures['START'].dt.to_period('M')
procedure_trends = procedures.groupby('Year-Month').size().reset_index(name='Procedure Count')
procedure_trends['Year-Month'] = procedure_trends['Year-Month'].astype(str)

# Plot the trend
fig = px.line(procedure_trends, x='Year-Month', y='Procedure Count',
              title='Monthly Procedure Volume',
              labels={'Year-Month': 'Year-Month', 'Procedure Count': 'Number of Procedures'},
              markers=True)
fig.update_layout(xaxis=dict(title='Year-Month', tickangle=45), yaxis_title='Procedure Count')
fig.show()


C:\Users\faisa\AppData\Local\Temp\ipykernel_15480\74845133.py:4: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



##  COVID_19_Test_to_Treat: Active Facilities by City

In [ ]:
# Count facilities by state
state_counts = COVID_19_Test_to_Treat['CITY'].value_counts().reset_index()
state_counts.columns = ['CITY', 'Facility Count']

# Plot the state distribution
fig = px.bar(state_counts, x='CITY', y='Facility Count',
             title='Active COVID-19 Test-to-Treat Facilities by City',
             labels={'CITY': 'City', 'Facility Count': 'Number of Facilities'},
             text='Facility Count')
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(xaxis_title='City', yaxis_title='Number of Facilities')
fig.show()


## Monthly COVID-19 Cases by County

In [ ]:
# Convert 'Date' to datetime format
Daily_Town_COVID19['Date'] = pd.to_datetime(Daily_Town_COVID19['Date'])

# Extract year and month from 'Date'
Daily_Town_COVID19['Year-Month'] = Daily_Town_COVID19['Date'].dt.to_period('M')

# Convert 'Year-Month' to string to avoid serialization issues
Daily_Town_COVID19['Year-Month'] = Daily_Town_COVID19['Year-Month'].astype(str)

# Group by Year-Month and COUNTY, summing up the daily cases
monthly_cases = Daily_Town_COVID19.groupby(['Year-Month', 'COUNTY'])['daily_cases'].sum().reset_index()

fig = px.bar(monthly_cases, x='Year-Month', y='daily_cases', color='COUNTY',
             title='Monthly COVID-19 Cases by COUNTY',
             labels={'Year-Month': 'Year-Month', 'daily_cases': 'Total Monthly Cases'},
             barmode='group')
fig.update_layout(xaxis=dict(tickformat='%Y-%m'), yaxis_title='Total Monthly Cases')
fig.show()




# Phi_K Correlation

## encounters correlation

In [ ]:
# Step 1: Convert START and STOP to datetime
encounters['START'] = pd.to_datetime(encounters['START'], errors='coerce')
encounters['STOP'] = pd.to_datetime(encounters['STOP'], errors='coerce')

# Step 2: Convert datetime columns to Unix timestamps
encounters['START'] = encounters['START'].astype('int64') // 10**9
encounters['STOP'] = encounters['STOP'].astype('int64') // 10**9

encounters_no_id = encounters.drop(columns=['Id'])

# Step 4: Compute the Phi_k correlation matrix
phik_matrix_encounters = encounters_no_id.phik_matrix()

print(phik_matrix_encounters)

interval columns not set, guessing: ['START', 'STOP', 'CODE', 'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'REASONCODE']


c:\Users\faisa\AppData\Local\Programs\Python\Python312\Lib\site-packages\phik\data_quality.py:82: UserWarning:

Not enough unique value for variable ORGANIZATION for analysis 1. Dropping this column



                        START      STOP   PATIENT     PAYER  ENCOUNTERCLASS      CODE  DESCRIPTION  BASE_ENCOUNTER_COST  TOTAL_CLAIM_COST  PAYER_COVERAGE  REASONCODE  REASONDESCRIPTION  Year-Month
START                1.000000  1.000000  0.708638  0.211796        0.221957  0.370403     0.606052             0.257186          0.373671        0.361538    0.023253           0.288309    0.999525
STOP                 1.000000  1.000000  0.708808  0.211734        0.221765  0.370598     0.605903             0.257358          0.374215        0.361535    0.023250           0.288111    0.999509
PATIENT              0.708638  0.708808  1.000000  0.946067        0.924871  0.904069     0.964284             0.875786          0.772137        0.888872    0.274789           0.970857    0.832955
PAYER                0.211796  0.211734  0.946067  1.000000        0.329815  0.317743     0.582656             0.294825          0.505541        0.826030    0.000000           0.415362    0.337322
ENCOUNTERCLASS 

In [ ]:
fig_encounters = go.Figure(
    data=go.Heatmap(
        z=phik_matrix_encounters.values,
        x=phik_matrix_encounters.columns,
        y=phik_matrix_encounters.index,
        colorscale="deep",
        colorbar=dict(title="Phi_k Correlation"),
        texttemplate="%{z:.5}" 
    )
)

fig_encounters.update_layout(
    title="Phi_k Correlation Matrix (encounters)",
    xaxis_title="Features",
    yaxis_title="Features",
    xaxis=dict(tickangle=90),
    autosize=True,
    margin=dict(l=150, r=150, t=50, b=150),  # Adjust margins for better visibility
    yaxis=dict(tickmode='array', tickvals=phik_matrix_encounters.index),  # Ensure all y-axis labels are visible
    width=1200,  # Set width to make it square
    height=800
      # Set height to make it square
)

fig_encounters.show()

Dropping Id: The Id column has too many unique values (27891), making it unsuitable for correlation analysis because it could lead to computational inefficiencies and may not provide meaningful insights.

Dropping ORGANIZATION: The ORGANIZATION column has only one value, making it unhelpful for the correlation analysis. Variables with limited variability often do not contribute useful information for correlation analysis.

Datetime Conversion: Converting START and STOP to Unix timestamps is necessary because correlation analysis works best with numeric data. By converting them into numeric format, they can be treated as interval variables. The warning about astype('int64') // 10**9 is noted, and an alternative using .view('int64') could be considered to avoid future warnings.

In [ ]:
# Revert Unix timestamps back to datetime
encounters['START'] = pd.to_datetime(encounters['START'], unit='s')
encounters['STOP'] = pd.to_datetime(encounters['STOP'], unit='s')

In [ ]:
encounters.head(1)

,Id,START,STOP,PATIENT,ORGANIZATION,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION,Year-Month
0,32c84703-2481-49cd-d571-3899d5820253,2011-01-02 09:26:36,2011-01-02 12:58:36,3de74169-7f67-9304-91d4-757e0f3a14d2,d78e84ec-30aa-3bba-a33a-f29a3a454662,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,185347001,Encounter for problem (procedure),85.55,1018.02,0.0,-1.0,None,2011-01


Reverting Unix Timestamps: After converting the START and STOP columns to Unix timestamps for correlation analysis, we revert them back to datetime format. This is done to ensure that the original datetime values are preserved for any future use, such as for interpretation, further analysis, or visualization where datetime format might be required.

## patients correlation

In [ ]:
patients_no_id_LL_S = patients.drop(columns=['Id','LAT','LON','SUFFIX'])
phik_matrix_patients = patients_no_id_LL_S.phik_matrix()

print(phik_matrix_patients)

interval columns not set, guessing: ['Age']


c:\Users\faisa\AppData\Local\Programs\Python\Python312\Lib\site-packages\phik\data_quality.py:82: UserWarning:

Not enough unique value for variable STATE for analysis 1. Dropping this column



            BIRTHDATE  DEATHDATE    PREFIX     FIRST      LAST    MAIDEN   MARITAL      RACE  ETHNICITY    GENDER  BIRTHPLACE   ADDRESS      CITY    COUNTY       ZIP       Age  Age Group
BIRTHDATE    1.000000   0.000000  0.923577  0.000000  0.975715  0.998890  0.460902  0.994649   0.993302  0.994391    0.000000  1.000000  0.998560  0.996913  0.985098  1.000000   0.858132
DEATHDATE    0.000000   1.000000  0.000000  0.995668  0.000000  0.000000  0.000000  0.364264   0.145632  0.060071    0.924111  1.000000  0.763291  0.778404  0.543096  0.000000   0.000000
PREFIX       0.923577   0.000000  1.000000  0.973172  0.000000  0.794249  0.454702  0.009834   0.049245  1.000000    0.360073  1.000000  0.057871  0.060976  0.078470  0.102937   0.101476
FIRST        0.000000   0.995668  0.973172  1.000000  0.000000  0.999611  0.000000  0.000000   0.673368  0.980090    0.000000  1.000000  0.798352  0.822523  0.000000  0.459113   0.000000
LAST         0.975715   0.000000  0.000000  0.000000  1.000000  0

In [ ]:
fig_patients = go.Figure(
    data=go.Heatmap(
        z=phik_matrix_patients.values,
        x=phik_matrix_patients.columns,
        y=phik_matrix_patients.index,
        colorscale="deep",
        colorbar=dict(title="Phi_k Correlation"),
        texttemplate="%{z:.5}"
        
    )
)

fig_patients.update_layout(
    title="Phi_k Correlation Matrix (patients)",
    xaxis_title="Features",
    yaxis_title="Features",
    xaxis=dict(tickangle=90),
    autosize=True,
    margin=dict(l=150, r=150, t=50, b=150),  # Adjust margins for better visibility
    yaxis=dict(tickmode='array', tickvals=phik_matrix_patients.index),  # Ensure all y-axis labels are visible
    width=1200,  # Set width to make it square
    height=800
)

fig_patients.show()

Dropping Id: The Id column has too many unique values (973), making it unsuitable for correlation analysis because it could lead to computational inefficiencies and may not provide meaningful insights.

Dropping LAT and LON: These are geographic coordinates, which do not contribute meaningfully to the correlation analysis.

Dropping SUFFIX: It contains mostly empty values that don’t contribute to our analysis (like academic titles), it won't offer much value in the context of demographic or healthcare-related insights.

Dropping STATE : This column has only one value, making it unhelpful for the correlation analysis. Variables with limited variability often do not contribute useful information for correlation analysis.


## procedures correlation

In [ ]:
print(procedures['BASE_COST'].nunique())


1


In [ ]:
print(procedures['BASE_COST'].value_counts())


BASE_COST
431    30988
Name: count, dtype: int64


In [ ]:
# Step 1: Convert START and STOP columns to datetime
procedures['START'] = pd.to_datetime(procedures['START'], errors='coerce')
procedures['STOP'] = pd.to_datetime(procedures['STOP'], errors='coerce')

# Step 2: Convert datetime columns to Unix timestamps (numeric format)
procedures['START'] = procedures['START'].astype('int64') // 10**9  # Convert to seconds
procedures['STOP'] = procedures['STOP'].astype('int64') // 10**9

# Ensure BASE_COST is numeric and handle missing values
#procedures['BASE_COST'] = pd.to_numeric(procedures['BASE_COST'], errors='coerce').fillna(0)

# Step 3: Define interval columns (exclude ENCOUNTER)
interval_columns = ['BASE_COST', 'START', 'STOP']

# Step 4: Compute the Phi_k correlation matrix
phik_matrix_procedures = procedures.phik_matrix(interval_cols=interval_columns)

# Step 5: Display the Phi_k correlation matrix
print(phik_matrix_procedures)


c:\Users\faisa\AppData\Local\Programs\Python\Python312\Lib\site-packages\phik\data_quality.py:59: UserWarning:

The number of unique values of variable ENCOUNTER is large: 7291. Are you sure this is not an interval variable? Analysis for pairs of variables including ENCOUNTER can be slow.

c:\Users\faisa\AppData\Local\Programs\Python\Python312\Lib\site-packages\phik\data_quality.py:72: UserWarning:

Not enough unique value for variable BASE_COST for analysis 1. Dropping this column



                      START      STOP   PATIENT  ENCOUNTER      CODE  DESCRIPTION  REASONCODE  REASONDESCRIPTION  Year-Month
START              1.000000  1.000000  0.829674   0.999991  0.438171     0.442280    0.206057           0.206057    0.999373
STOP               1.000000  1.000000  0.829663   0.999990  0.438240     0.442348    0.206329           0.206329    0.999373
PATIENT            0.829674  0.829663  1.000000   1.000000  0.953113     0.955520    0.917441           0.917441    0.980758
ENCOUNTER          0.999991  0.999990  1.000000   1.000000  0.993126     0.994098    0.992406           0.992406    0.999981
CODE               0.438171  0.438240  0.953113   0.993126  1.000000     1.000000    0.981907           0.981907    0.655939
DESCRIPTION        0.442280  0.442348  0.955520   0.994098  1.000000     1.000000    0.983482           0.983482    0.653459
REASONCODE         0.206057  0.206329  0.917441   0.992406  0.981907     0.983482    1.000000           1.000000    0.432192


In [ ]:
fig_procedures = go.Figure(
    data=go.Heatmap(
        z=phik_matrix_procedures.values,
        x=phik_matrix_procedures.columns,
        y=phik_matrix_procedures.index,
        colorscale="deep",
        colorbar=dict(title="Phi_k Correlation"),
        texttemplate="%{z:.5}", 
    )
)

fig_procedures.update_layout(
    title="Phi_k Correlation Matrix (procedures)",
    xaxis_title="Features",
    yaxis_title="Features",
    xaxis=dict(tickangle=90),
    autosize=True,
    width=1200,  # Set width to make it square
    height=800
)

fig_procedures.show()

In [ ]:
# Revert Unix timestamps back to datetime
procedures['START'] = pd.to_datetime(procedures['START'], unit='s')
procedures['STOP'] = pd.to_datetime(procedures['STOP'], unit='s')

In [ ]:
procedures.head(1)

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,REASONCODE,REASONDESCRIPTION,Year-Month
13,2011-01-12 18:09:27,2011-01-12 19:05:48,712e95dd-f313-39c7-1fc9-8463c15c105f,44057ea7-178f-c3c3-965a-78e4521fa43b,710824005,Assessment of health and social care needs (pr...,431,-1.0,None,2011-01


Reverting Unix Timestamps: After converting the START and STOP columns to Unix timestamps for correlation analysis, we revert them back to datetime format. This is done to ensure that the original datetime values are preserved for any future use, such as for interpretation, further analysis, or visualization where datetime format might be required.

## COVID_19_Test_to_Treat Correlation

In [ ]:
phik_matrix_covid19 = COVID_19_Test_to_Treat.phik_matrix()

print(phik_matrix_covid19)

interval columns not set, guessing: ['Zip']


c:\Users\faisa\AppData\Local\Programs\Python\Python312\Lib\site-packages\phik\data_quality.py:82: UserWarning:

Not enough unique value for variable STATE for analysis 1. Dropping this column



                  Provider Name  Address1      CITY       Zip  Last Report Date  Geopoint
Provider Name          1.000000  0.997533  0.975642  1.000000          1.000000  0.996442
Address1               0.997533  1.000000  1.000000  0.949223          1.000000  0.999849
CITY                   0.975642  1.000000  1.000000  0.826859          0.000000  1.000000
Zip                    1.000000  0.949223  0.826859  1.000000          0.213606  0.929840
Last Report Date       1.000000  1.000000  0.000000  0.213606          1.000000  0.000000
Geopoint               0.996442  0.999849  1.000000  0.929840          0.000000  1.000000


In [ ]:
fig_covid19 = go.Figure(
    data=go.Heatmap(
        z=phik_matrix_covid19.values,
        x=phik_matrix_covid19.columns,
        y=phik_matrix_covid19.index,
        colorscale="deep",
        colorbar=dict(title="Phi_k Correlation"),
        texttemplate="%{z:.5}" 
    )
)

fig_covid19.update_layout(
    title="Phi_k Correlation Matrix (COVID_19_Test_to_Treat)",
    xaxis_title="Features",
    yaxis_title="Features",
    xaxis=dict(tickangle=90),
    autosize=True,
    margin=dict(l=150, r=150, t=50, b=150),  # Adjust margins for better visibility
    yaxis=dict(tickmode='array', tickvals=phik_matrix_covid19.index),  # Ensure all y-axis labels are visible
    width=1200,  # Set width to make it square
    height=800
      # Set height to make it square
)

fig_covid19.show()

## Daily_Town_COVID19 Correlation

In [ ]:
phik_matrix_daily = Daily_Town_COVID19.phik_matrix()

print(phik_matrix_daily)

interval columns not set, guessing: ['covid_cases_cumulative', 'daily_cases']
                            Date    COUNTY      Town  covid_cases_cumulative  daily_cases  Year-Month
Date                    1.000000  0.000000  0.000000                0.115548     0.264284    1.000000
COUNTY                  0.000000  1.000000  1.000000                0.485824     0.243903    0.000000
Town                    0.000000  1.000000  1.000000                0.718828     0.562494    0.000000
covid_cases_cumulative  0.115548  0.485824  0.718828                1.000000     0.685239    0.224015
daily_cases             0.264284  0.243903  0.562494                0.685239     1.000000    0.137375
Year-Month              1.000000  0.000000  0.000000                0.224015     0.137375    1.000000


In [ ]:
fig_daily = go.Figure(
    data=go.Heatmap(
        z=phik_matrix_daily.values,
        x=phik_matrix_daily.columns,
        y=phik_matrix_daily.index,
        colorscale="deep",
        colorbar=dict(title="Phi_k Correlation"),
        texttemplate="%{z:.5}" 
    )
)

fig_daily.update_layout(
    title="Phi_k Correlation Matrix (Daily_Town_COVID19)",
    xaxis_title="Features",
    yaxis_title="Features",
    xaxis=dict(tickangle=90),
    autosize=True,
    margin=dict(l=150, r=150, t=50, b=150),  # Adjust margins for better visibility
    yaxis=dict(tickmode='array', tickvals=phik_matrix_daily.index),  # Ensure all y-axis labels are visible
    width=1200,  # Set width to make it square
    height=800
      # Set height to make it square
)

fig_daily.show()

# Dependency Discovery (UCC, FD, IND)

## Unique Column Combinations (UCC)

Using hierarchical clustering of attributes (HCA) to identify unique sets of columns. HCA: A hybrid algorithm designed for detecting UCCs efficiently by combining various strategies like pruning and sampling.

In [ ]:
# Define a function to check if a column combination is unique
def is_unique(data, cols):
    return data.duplicated(subset=cols).sum() == 0

# Define the HCA function
def hca_ucc(data):
    cols = data.columns.tolist()
    uccs = []
    for r in range(1, len(cols) + 1):
        for comb in combinations(cols, r):
            if is_unique(data, comb):
                uccs.append(comb)
                # Prune supersets of this combination
                cols = [col for col in cols if col not in comb]
                break
    return uccs

# Load datasets
datasets = {
    "encounters": encounters,
    "patients": patients,
    "payers": payers,
    "procedures": procedures,
    "COVID_19_Test_to_Treat": COVID_19_Test_to_Treat,
    "Daily_Town_COVID19":Daily_Town_COVID19
}

# Run HCA on all datasets
ucc_results = {}
for name, df in datasets.items():
    print(f"Processing {name}...")
    ucc_results[name] = hca_ucc(df)

# Display results
for name, uccs in ucc_results.items():
    print(f"\nDataset: {name}")
    print(f"Unique Column Combinations: {uccs}")


Processing encounters...
Processing patients...
Processing payers...
Processing procedures...
Processing COVID_19_Test_to_Treat...
Processing Daily_Town_COVID19...

Dataset: encounters
Unique Column Combinations: [('Id',), ('START', 'PATIENT', 'CODE')]

Dataset: patients
Unique Column Combinations: [('Id',), ('BIRTHDATE', 'FIRST'), ('DEATHDATE', 'PREFIX', 'ADDRESS'), ('LAST', 'SUFFIX', 'MAIDEN', 'LAT'), ('MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'LON')]

Dataset: payers
Unique Column Combinations: [('Id',), ('NAME', 'ADDRESS')]

Dataset: procedures
Unique Column Combinations: [('START', 'STOP', 'CODE')]

Dataset: COVID_19_Test_to_Treat
Unique Column Combinations: [('Provider Name', 'Geopoint')]

Dataset: Daily_Town_COVID19
Unique Column Combinations: [('Date', 'Town')]


### Analysis of Results

1. **Encounters:**
   - `('Id',)`: This indicates that the `Id` column itself is a unique identifier for encounters, which is expected in many datasets.
   - `('START', 'PATIENT', 'CODE')`: This combination suggests that an encounter is uniquely defined by the start time, the patient involved, and the specific code of the encounter.

2. **Patients:**
   - Several combinations indicate the diversity in how patient data can be uniquely identified, not just by `Id` but also by combinations of demographic details.

3. **Payers:**
   - `('Id',)`: Standard for an identifier.
   - `('NAME', 'ADDRESS')`: This combination might be especially interesting if you're looking into data normalization or identifying duplicate entries in your data.

4. **Procedures:**
   - `('START', 'STOP', 'CODE')`: Indicates that procedures are uniquely defined by their start and stop times alongside the procedure code.

5. **COVID_19_Test_to_Treat:**
   - `('Provider Name', 'Geopoint')`: Useful for identifying unique testing locations and providers.

6. **Daily_Town_COVID19:**
   - `('Date', 'Town')`: A straightforward combination that uniquely identifies records by date and town.


### Duplicate Detection and Removal

In [ ]:
# Function to remove duplicates based on identified UCCs
def remove_duplicates(df, uccs, dataset_name):
    for ucc in uccs:
        initial_count = len(df)
        df.drop_duplicates(subset=ucc, inplace=True)
        final_count = len(df)
        print(f"{dataset_name}: Removed {initial_count - final_count} duplicates using UCC {ucc}")

# Dictionary of datasets and their UCCs
datasets = {
    "encounters": (encounters, [('Id',), ('START', 'PATIENT', 'CODE')]),
    "patients": (patients, [('Id',), ('BIRTHDATE', 'FIRST'), ('DEATHDATE', 'PREFIX', 'ADDRESS'), ('LAST', 'SUFFIX', 'MAIDEN', 'LAT'), ('MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'LON')]),
    "payers": (payers, [('Id',), ('NAME', 'ADDRESS')]),
    "procedures": (procedures, [('START', 'STOP', 'CODE')]),
    "COVID_19_Test_to_Treat": (COVID_19_Test_to_Treat, [('Provider Name', 'Geopoint')]),
    "Daily_Town_COVID19": (Daily_Town_COVID19, [('Date', 'Town')])
}

# Process each dataset
for dataset_name, (df, uccs) in datasets.items():
    print(f"Processing {dataset_name}...")
    remove_duplicates(df, uccs, dataset_name)


Processing encounters...
encounters: Removed 0 duplicates using UCC ('Id',)
encounters: Removed 0 duplicates using UCC ('START', 'PATIENT', 'CODE')
Processing patients...
patients: Removed 0 duplicates using UCC ('Id',)
patients: Removed 0 duplicates using UCC ('BIRTHDATE', 'FIRST')
patients: Removed 0 duplicates using UCC ('DEATHDATE', 'PREFIX', 'ADDRESS')
patients: Removed 0 duplicates using UCC ('LAST', 'SUFFIX', 'MAIDEN', 'LAT')
patients: Removed 0 duplicates using UCC ('MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'LON')
Processing payers...
payers: Removed 0 duplicates using UCC ('Id',)
payers: Removed 0 duplicates using UCC ('NAME', 'ADDRESS')
Processing procedures...
procedures: Removed 0 duplicates using UCC ('START', 'STOP', 'CODE')
Processing COVID_19_Test_to_Treat...
COVID_19_Test_to_Treat: Removed 0 duplicates using UCC ('Provider Name', 'Geopoint')
Processing Daily_Town_COVID19...
Daily_Town_COVID19: Removed 0 duplicates using UCC ('Date', 'Town')


### Data Validation

After cleaning up duplicates, it’s crucial to validate our datasets remain consistent and that the removal hasn't inadvertently affected the integrity of our data.

In [ ]:
def verify_uccs(df, uccs, dataset_name):
    for ucc in uccs:
        if df.duplicated(subset=ucc).any():
            print(f"{dataset_name} still has duplicates on UCC {ucc}")
        else:
            print(f"{dataset_name}: No duplicates found for UCC {ucc} post-cleanup")

# Dictionary of datasets for post-cleanup validation
datasets = {
    "encounters": (encounters, [('Id',), ('START', 'PATIENT', 'CODE')]),
    "patients": (patients, [('Id',), ('BIRTHDATE', 'FIRST'), ('DEATHDATE', 'PREFIX', 'ADDRESS'), ('LAST', 'SUFFIX', 'MAIDEN', 'LAT'), ('MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'LON')]),
    "payers": (payers, [('Id',), ('NAME', 'ADDRESS')]),
    "procedures": (procedures, [('START', 'STOP', 'CODE')]),
    "COVID_19_Test_to_Treat": (COVID_19_Test_to_Treat, [('Provider Name', 'Geopoint')]),
    "Daily_Town_COVID19": (Daily_Town_COVID19, [('Date', 'Town')])
}

# Run post-cleanup UCC verification
for dataset_name, (df, uccs) in datasets.items():
    verify_uccs(df, uccs, dataset_name)


encounters: No duplicates found for UCC ('Id',) post-cleanup
encounters: No duplicates found for UCC ('START', 'PATIENT', 'CODE') post-cleanup
patients: No duplicates found for UCC ('Id',) post-cleanup
patients: No duplicates found for UCC ('BIRTHDATE', 'FIRST') post-cleanup
patients: No duplicates found for UCC ('DEATHDATE', 'PREFIX', 'ADDRESS') post-cleanup
patients: No duplicates found for UCC ('LAST', 'SUFFIX', 'MAIDEN', 'LAT') post-cleanup
patients: No duplicates found for UCC ('MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'LON') post-cleanup
payers: No duplicates found for UCC ('Id',) post-cleanup
payers: No duplicates found for UCC ('NAME', 'ADDRESS') post-cleanup
procedures: No duplicates found for UCC ('START', 'STOP', 'CODE') post-cleanup
COVID_19_Test_to_Treat: No duplicates found for UCC ('Provider Name', 'Geopoint') post-cleanup
Daily_Town_COVID19: No duplicates found for UCC ('Date', 'Town') post-cleanup


In [ ]:
# Print all column names for each dataset
for dataset_name, (df, _) in datasets.items():
    print(f"{dataset_name} columns: {df.columns.tolist()}")

encounters columns: ['Id', 'START', 'STOP', 'PATIENT', 'ORGANIZATION', 'PAYER', 'ENCOUNTERCLASS', 'CODE', 'DESCRIPTION', 'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'REASONCODE', 'REASONDESCRIPTION', 'Year-Month']
patients columns: ['Id', 'BIRTHDATE', 'DEATHDATE', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP', 'LAT', 'LON', 'Age', 'Age Group']
payers columns: ['Id', 'NAME', 'ADDRESS', 'CITY', 'STATE_HEADQUARTERED', 'ZIP', 'PHONE']
procedures columns: ['START', 'STOP', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION', 'BASE_COST', 'REASONCODE', 'REASONDESCRIPTION', 'Year-Month']
COVID_19_Test_to_Treat columns: ['Provider Name', 'Address1', 'CITY', 'STATE', 'Zip', 'Last Report Date', 'Geopoint']
Daily_Town_COVID19 columns: ['Date', 'COUNTY', 'Town', 'covid_cases_cumulative', 'daily_cases', 'Year-Month']


## Functional Dependency (FD)

Using A-priori algorithm helps determine which sets of columns (or attributes) functionally determine other columns within the dataset.

Direct Establishment of FDs from UCCs: The code leverages Unique Column Combinations (UCCs) to directly establish functional dependencies without further validation. If a set of columns is a UCC, then it is known to uniquely identify all records, and thus, it functionally determines all other columns in the dataset. This reduces the computational load by bypassing unnecessary checks for these combinations. Exclusion of UCC Columns in Further Tests: After establishing FDs from the UCCs, the script excludes these columns from further tests to find new FDs. This pruning strategy avoids redundant computations and focuses on potential dependencies that involve non-UCC columns.

Efficient Data Grouping,  instead of grouping data by all determinant columns at once (which can be memory-intensive and slower), the script employs an incremental grouping approach. For determinants involving multiple columns, it first groups by a subset of these columns and then progressively refines the grouping by incorporating additional columns. This approach can significantly reduce memory usage and improve execution speed by managing smaller subsets of data at each step.

Contextual Warning Management, within the incremental grouping logic, runtime warnings related to numerical operations (such as division by zero) are suppressed to avoid log clutter and focus on meaningful outputs. 

In [ ]:
# Suppress all warnings, including FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

def find_fd(data, uccs, min_confidence=0.9):
    fds = []
    all_columns = list(data.columns)
    
    # Pruning: Directly establish FDs from UCCs
    # This is based on the principle that a unique column combination (UCC) functionally determines all other columns.
    for ucc in uccs:
        for col in all_columns:
            if col not in ucc:
                fds.append((ucc, col))

    # Function to check if a determinant determines a dependent incrementally
    def is_fd(determinant, dependent):
        # Handle grouping incrementally with proper handling of the observed parameter
        if len(determinant) > 1:
            # Intermediate grouping with observed parameter
            intermediate_group = data.groupby(list(determinant[:-1]), observed=True)[dependent].nunique()
            # Final grouping refinement with observed parameter
            final_group = intermediate_group.groupby(data[determinant[-1]], observed=True).nunique()
        else:
            # Single column grouping with observed parameter
            final_group = data.groupby(list(determinant), observed=True)[dependent].nunique()

        # Pruning: Generate combinations of columns not in UCCs to avoid redundant checks.
        total_groups = len(final_group)
        confidence = (final_group == 1).sum() / total_groups
        return confidence >= min_confidence

    # Pruning: Generate combinations of columns not in UCCs to avoid redundant checks.
    non_ucc_columns = [col for col in all_columns if all(col not in ucc for ucc in uccs)]
    for r in range(1, len(non_ucc_columns) + 1):
        for combo in combinations(non_ucc_columns, r):
            remaining_columns = [col for col in all_columns if col not in combo]
            for dep in remaining_columns:
                if is_fd(combo, dep):
                    fds.append((combo, dep))

    return fds

# Example dataset application
datasets = {
    "encounters": encounters,
    "patients": patients,
    "payers": payers,
    "procedures": procedures,
    "COVID_19_Test_to_Treat": COVID_19_Test_to_Treat,
    "Daily_Town_COVID19": Daily_Town_COVID19
}

# UCCs from your previous analysis
uccs_dict = {
    'encounters': [('Id',), ('START', 'PATIENT', 'CODE')],
    'patients': [('Id',), ('BIRTHDATE', 'FIRST'), ('DEATHDATE', 'PREFIX', 'ADDRESS'), ('LAST', 'SUFFIX', 'MAIDEN', 'LAT'), ('MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'LON')],
    'payers': [('Id',), ('NAME', 'ADDRESS')],
    'procedures': [('START', 'STOP', 'CODE')],
    'COVID_19_Test_to_Treat': [('Provider Name', 'Geopoint')],
    'Daily_Town_COVID19': [('Date', 'Town')]
}

# Process each dataset
for dataset_name, df in datasets.items():
    print(f"Analyzing {dataset_name}")
    uccs = uccs_dict[dataset_name]
    fds = find_fd(df, uccs, min_confidence=0.95)
    for fd in fds:
        print(f"Determinant {fd[0]} functionally determines {fd[1]}")
    print("\n")


Analyzing encounters
Determinant ('Id',) functionally determines START
Determinant ('Id',) functionally determines STOP
Determinant ('Id',) functionally determines PATIENT
Determinant ('Id',) functionally determines ORGANIZATION
Determinant ('Id',) functionally determines PAYER
Determinant ('Id',) functionally determines ENCOUNTERCLASS
Determinant ('Id',) functionally determines CODE
Determinant ('Id',) functionally determines DESCRIPTION
Determinant ('Id',) functionally determines BASE_ENCOUNTER_COST
Determinant ('Id',) functionally determines TOTAL_CLAIM_COST
Determinant ('Id',) functionally determines PAYER_COVERAGE
Determinant ('Id',) functionally determines REASONCODE
Determinant ('Id',) functionally determines REASONDESCRIPTION
Determinant ('Id',) functionally determines Year-Month
Determinant ('START', 'PATIENT', 'CODE') functionally determines Id
Determinant ('START', 'PATIENT', 'CODE') functionally determines STOP
Determinant ('START', 'PATIENT', 'CODE') functionally determine

### Validate the FD

In [ ]:
def validate_fds(data, fds):
    violations = {}
    for determinant, dependent in fds:
        # Group by the determinant and count unique values in the dependent column
        group_counts = data.groupby(list(determinant))[dependent].nunique()
        # Identify where more than one unique value exists for the dependent variable
        if any(group_counts > 1):
            violations[(determinant, dependent)] = group_counts[group_counts > 1].count()
    return violations

# Example usage with a dataset and identified FDs
for dataset_name, df in datasets.items():
    print(f"Validating FDs in {dataset_name}")
    uccs = uccs_dict[dataset_name]
    fd_violations = validate_fds(df, [(ucc, col) for ucc in uccs for col in df.columns if col not in ucc])
    if fd_violations:
        print(f"FD violations found in {dataset_name}: {fd_violations}")
    else:
        print(f"No FD violations in {dataset_name}")
    print("\n")


Validating FDs in encounters
No FD violations in encounters


Validating FDs in patients
No FD violations in patients


Validating FDs in payers
No FD violations in payers


Validating FDs in procedures
No FD violations in procedures


Validating FDs in COVID_19_Test_to_Treat
No FD violations in COVID_19_Test_to_Treat


Validating FDs in Daily_Town_COVID19
No FD violations in Daily_Town_COVID19




With no FD violations, the data is in a strong position for more advanced analyses and applications. Proceeding with IND analysis and integrating.

## Inclusion Dependencies (IND)

Using MIND algorithm to efficiently detect inclusion dependencies between columns across different tables. It operates by iteratively testing smaller candidate sets of column combinations and expanding these candidates based on their ability to maintain the inclusion dependency property.

In [ ]:
def precompute_stats(datasets):
    """ Precompute statistics for each dataset which includes data types and unique value counts. """
    stats = {}
    for name, df in datasets.items():
        stats[name] = {
            col: {'unique_count': df[col].nunique(), 'data_type': df[col].dtype}
            for col in df.columns
        }
    return stats

def check_inclusion(df1, col1, df2, col2):
    """ Check if all elements in df1[col1] are included in df2[col2]. Optimized by checking unique values. """
    unique_values_1 = pd.Series(df1[col1].dropna().unique())
    unique_values_2 = pd.Series(df2[col2].dropna().unique())
    return unique_values_1.isin(unique_values_2).all()

def find_mind_like_inds(datasets, stats):
    """ A function that implements a MIND-like algorithm to discover INDs with advanced pruning. """
    inds = []
    # Loop through each pair of datasets
    keys = list(datasets.keys())
    for key1 in keys:
        for key2 in keys:
            if key1 != key2:
                df1 = datasets[key1]
                df2 = datasets[key2]
                for col1 in df1.columns:
                    for col2 in df2.columns:
                        # Prune based on data type and unique count heuristic
                        if (stats[key1][col1]['data_type'] == stats[key2][col2]['data_type'] and
                                stats[key1][col1]['unique_count'] <= stats[key2][col2]['unique_count']):
                            if check_inclusion(df1, col1, df2, col2):
                                inds.append(f"{key1}.{col1} -> {key2}.{col2}")
    return inds

datasets = {
    "encounters": encounters,
    "patients": patients,
    "payers": payers,
    "procedures": procedures,
    "COVID_19_Test_to_Treat": COVID_19_Test_to_Treat,
    "Daily_Town_COVID19": Daily_Town_COVID19
}

# Precompute statistics for each column in each dataset
stats = precompute_stats(datasets)

# Find inclusion dependencies using the advanced MIND-like algorithm
identified_inds = find_mind_like_inds(datasets, stats)
for ind in identified_inds:
    print(ind)


encounters.PAYER -> payers.Id
patients.STATE -> COVID_19_Test_to_Treat.STATE
patients.COUNTY -> Daily_Town_COVID19.COUNTY
payers.Id -> encounters.PAYER
procedures.Year-Month -> encounters.Year-Month
procedures.BASE_COST -> Daily_Town_COVID19.covid_cases_cumulative
COVID_19_Test_to_Treat.CITY -> patients.CITY
COVID_19_Test_to_Treat.STATE -> patients.STATE
COVID_19_Test_to_Treat.CITY -> Daily_Town_COVID19.Town
Daily_Town_COVID19.COUNTY -> patients.COUNTY


### Validate the IND

In [ ]:
def check_all_column_relationships(datasets):
    """ Checks every column in each dataset against every column in every other dataset for common values and counts unique values. """
    relationships = {}  # Store results

    keys = list(datasets.keys())
    for i, key1 in enumerate(keys):
        for j, key2 in enumerate(keys):
            if i != j:  # Avoid comparing the same dataset
                df1 = datasets[key1]
                df2 = datasets[key2]
                for col1 in df1.columns:
                    for col2 in df2.columns:
                        # Compare columns across datasets
                        unique_values_1 = set(df1[col1].dropna().astype(str).unique())
                        unique_values_2 = set(df2[col2].dropna().astype(str).unique())
                        intersection = unique_values_1.intersection(unique_values_2)
                        if intersection:  # If there are common values
                            relationships[f"{key1}.{col1} -> {key2}.{col2}"] = {
                                'common_values_count': len(intersection),
                                'total_unique_values_col1': len(unique_values_1),
                                'total_unique_values_col2': len(unique_values_2),
                                #'example_values': list(intersection)[:5]  # Provide some examples for inspection
                            }
    return relationships

# Assume datasets dictionary is defined with DataFrames 'encounters', 'patients', etc.
datasets = {
    "encounters": encounters,
    "patients": patients,
    "payers": payers,
    "procedures": procedures,
    "COVID_19_Test_to_Treat": COVID_19_Test_to_Treat,
    "Daily_Town_COVID19": Daily_Town_COVID19
}

# Run the function and print the results
all_relationships = check_all_column_relationships(datasets)
for relationship, details in all_relationships.items():
    print(f"{relationship} has {details['common_values_count']} common values; " +
          f"Column1 has {details['total_unique_values_col1']} unique values; " +
          f"Column2 has {details['total_unique_values_col2']} unique values.")


encounters.PATIENT -> patients.Id has 910 common values; Column1 has 911 unique values; Column2 has 973 unique values.
encounters.PAYER -> payers.Id has 10 common values; Column1 has 10 unique values; Column2 has 10 unique values.
encounters.Id -> procedures.ENCOUNTER has 3376 common values; Column1 has 18183 unique values; Column2 has 7291 unique values.
encounters.START -> procedures.START has 2945 common values; Column1 has 17973 unique values; Column2 has 30358 unique values.
encounters.START -> procedures.STOP has 1 common values; Column1 has 17973 unique values; Column2 has 30556 unique values.
encounters.STOP -> procedures.START has 9 common values; Column1 has 18141 unique values; Column2 has 30358 unique values.
encounters.STOP -> procedures.STOP has 581 common values; Column1 has 18141 unique values; Column2 has 30556 unique values.
encounters.PATIENT -> procedures.PATIENT has 664 common values; Column1 has 911 unique values; Column2 has 697 unique values.
encounters.REASONCO

### Analyzing Inclusion Dependencies (about the previous results):

**Inclusion Dependency is Valid:**

- encounters.PAYER -> payers.Id has 10 common values; Column1 has 10 unique values; Column2 has 10 unique values.
- patients.STATE -> COVID_19_Test_to_Treat.STATE has 1 common values; Column1 has 1 unique values; Column2 has 1 unique values.
- patients.COUNTY -> Daily_Town_COVID19.COUNTY has 5 common values; Column1 has 5 unique values; Column2 has 5 unique values. 
- payers.Id -> encounters.PAYER has 10 common values; Column1 has 10 unique values; Column2 has 10 unique values.
- procedures.Year-Month -> encounters.Year-Month has 133 common values; Column1 has 133 unique values; Column2 has 134 unique values. 
- COVID_19_Test_to_Treat.CITY -> patients.CITY has 27 common values; Column1 has 27 unique values; Column2 has 29 unique values.
- COVID_19_Test_to_Treat.STATE -> patients.STATE has 1 common values; Column1 has 1 unique values; Column2 has 1 unique values.
- COVID_19_Test_to_Treat.CITY -> Daily_Town_COVID19.Town has 27 common values; Column1 has 27 unique values; Column2 has 147 unique values. 
- Daily_Town_COVID19.COUNTY -> patients.COUNTY has 5 common values; Column1 has 5 unique values; Column2 has 5 unique values.
- Daily_Town_COVID19.Year-Month -> encounters.Year-Month has 10 common values; Column1 has 10 unique values; Column2 has 134 unique values. 
- Daily_Town_COVID19.Year-Month -> procedures.Year-Month has 10 common values; Column1 has 10 unique values; Column2 has 133 unique values.


**Inclusion Dependency is Not Valid:**

- encounters.Year-Month -> procedures.Year-Month has 133 common values; Column1 has 134 unique values; Column2 has 133 unique values.
- encounters.Year-Month -> Daily_Town_COVID19.Year-Month has 10 common values; Column1 has 134 unique values; Column2 has 10 unique values.
- encounters.PATIENT -> patients.Id has 910 common values; Column1 has 911 unique values; Column2 has 973 unique values. 
- encounters.Id -> procedures.ENCOUNTER has 3376 common values; Column1 has 18183 unique values; Column2 has 7291 unique values. 
- encounters.START -> procedures.START has 2945 common values; Column1 has 17973 unique values; Column2 has 30358 unique values. 
- encounters.START -> procedures.STOP has 1 common values; Column1 has 17973 unique values; Column2 has 30556 unique values. 
- encounters.STOP -> procedures.START has 9 common values; Column1 has 18141 unique values; Column2 has 30358 unique values. 
- encounters.STOP -> procedures.STOP has 581 common values; Column1 has 18141 unique values; Column2 has 30556 unique values. 
- encounters.PATIENT -> procedures.PATIENT has 664 common values; Column1 has 911 unique values; Column2 has 697 unique values. 
- encounters.REASONCODE -> procedures.REASONCODE has 14 common values; Column1 has 66 unique values; Column2 has 18 unique values. 
- encounters.REASONDESCRIPTION -> procedures.REASONDESCRIPTION has 14 common values; Column1 has 66 unique values; Column2 has 18 unique values.
- patients.CITY -> COVID_19_Test_to_Treat.CITY has 27 common values; Column1 has 29 unique values; Column2 has 27 unique values. 
- patients.Id -> procedures.PATIENT has 696 common values; Column1 has 973 unique values; Column2 has 697 unique values. 
- patients.CITY -> Daily_Town_COVID19.Town has 28 common values; Column1 has 29 unique values; Column2 has 147 unique values. 
- patients.Id -> encounters.PATIENT has 910 common values; Column1 has 973 unique values; Column2 has 911 unique values. 
- patients.SUFFIX -> payers.STATE_HEADQUARTERED has 1 common values; Column1 has 4 unique values; Column2 has 7 unique values. 
- patients.ZIP -> payers.ZIP has 1 common values; Column1 has 71 unique values; Column2 has 8 unique values. 
- patients.CITY -> COVID_19_Test_to_Treat.CITY has 27 common values; Column1 has 29 unique values; Column2 has 27 unique values.
- patients.DEATHDATE -> Daily_Town_COVID19.Date has 17 common values; Column1 has 148 unique values; Column2 has 291 unique values.
- patients.COUNTY -> Daily_Town_COVID19.Town has 3 common values; Column1 has 5 unique values; Column2 has 147 unique values. 
- payers.STATE_HEADQUARTERED -> patients.SUFFIX has 1 common values; Column1 has 7 unique values; Column2 has 4 unique values. 
- payers.ZIP -> patients.ZIP has 1 common values; Column1 has 8 unique values; Column2 has 71 unique values. 
- procedures.Year-Month -> Daily_Town_COVID19.Year-Month has 10 common values; Column1 has 133 unique values; Column2 has 10 unique values.
- procedures.PATIENT -> patients.Id has 696 common values; Column1 has 697 unique values; Column2 has 973 unique values. 
- procedures.START -> encounters.START has 2945 common values; Column1 has 30358 unique values; Column2 has 17973 unique values. 
- procedures.START -> encounters.STOP has 9 common values; Column1 has 30358 unique values; Column2 has 18141 unique values.
- procedures.STOP -> encounters.START has 1 common values; Column1 has 30556 unique values; Column2 has 17973 unique values.
- procedures.STOP -> encounters.STOP has 581 common values; Column1 has 30556 unique values; Column2 has 18141 unique values.
- procedures.PATIENT -> encounters.PATIENT has 664 common values; Column1 has 697 unique values; Column2 has 911 unique values. 
- procedures.ENCOUNTER -> encounters.Id has 3376 common values; Column1 has 7291 unique values; Column2 has 18183 unique values.
- procedures.REASONCODE -> encounters.REASONCODE has 14 common values; Column1 has 18 unique values; Column2 has 66 unique values. 
- procedures.REASONDESCRIPTION -> encounters.REASONDESCRIPTION has 14 common values; Column1 has 18 unique values; Column2 has 66 unique values. 
- procedures.Year-Month -> Daily_Town_COVID19.Year-Month has 10 common values; Column1 has 133 unique values; Column2 has 10 unique values. COVID_19_Test_to_Treat.Zip -> Daily_Town_COVID19.covid_cases_cumulative has 26 common values; Column1 has 161 unique values; Column2 has 4222 unique values. 
- Daily_Town_COVID19.covid_cases_cumulative -> patients.Age has 70 common values; Column1 has 4222 unique values; Column2 has 70 unique values. 
- Daily_Town_COVID19.Town -> COVID_19_Test_to_Treat.CITY has 27 common values; Column1 has 147 unique values; Column2 has 27 unique values.
- Daily_Town_COVID19.covid_cases_cumulative -> procedures.BASE_COST has 1 common values; Column1 has 4222 unique values; Column2 has 1 unique values. 
- Daily_Town_COVID19.Town -> patients.CITY has 28 common values; Column1 has 147 unique values; Column2 has 29 unique values. 
- Daily_Town_COVID19.Date -> patients.DEATHDATE has 17 common values; Column1 has 291 unique values; Column2 has 148 unique values.
- Daily_Town_COVID19.Town -> patients.COUNTY has 3 common values; Column1 has 147 unique values; Column2 has 5 unique values. 
- Daily_Town_COVID19.covid_cases_cumulative -> COVID_19_Test_to_Treat.Zip has 26 common values; Column1 has 4222 unique values; Column2 has 161 unique values.


**Inclusion Dependency Is Rejected Despite Appearing Valid:**

- patients.Age -> Daily_Town_COVID19.covid_cases_cumulative has 70 common values; Column1 has 70 unique values; Column2 has 4222 unique values. 
- patients.Age -> Daily_Town_COVID19.daily_cases has 70 common values; Column1 has 70 unique values; Column2 has 390 unique values. 
- procedures.BASE_COST -> Daily_Town_COVID19.covid_cases_cumulative has 1 common values; Column1 has 1 unique values; Column2 has 4222 unique values.
- Daily_Town_COVID19.daily_cases -> patients.Age has 70 common values; Column1 has 390 unique values; Column2 has 70 unique values.

### Insight into the results:

Column 1 -> Column 2

Column 1 (dependent column) contains Foreign Key (FK)

Column 2 (referenced column)contains Primary Key (PK)


#### When Inclusion Dependency is Valid:
1. **All Values Match**: Every unique value in column 1  must also be found in column 2. The relationship is valid when there is a perfect match of all values in column 1 within column 2.

   **Example**:
   - **encounters.PAYER -> payers.Id**: All 10 unique values in encounters matched exactly with those in payers (10 of 10).

2. **Subset Relationship**: This is confirmed even if column 2 contains more values than column 1, as long as every value in column 1  appears in column 2. 

    **Example**:
    - **procedures.Year-Month -> encounters.Year-Month**:  All 133 unique values in procedures matched exactly with those in encounters (133 of 134).

#### When Inclusion Dependency is Not Valid:
 **Missing Values**: If even one value from column 1 is not found in column 2, the subset relationship does not hold, and therefore, the inclusion dependency is invalid.

   **Examples of Invalid INDs**:
   - **encounters.PATIENT -> patients.Id**: 910 of 911 unique values in encounters were found in patients, missing one value.

#### When Inclusion Dependency Is Rejected Despite Appearing Valid
 **Wrong Columns**: Initially, it may appear that all values match or there is a subset relationship, but if the columns are unrelated or their relationship does not make sense, we should disregard this apparent dependency.

   **Example**:    
   - **procedures.BASE_COST -> Daily_Town_COVID19.covid_cases_cumulative**: Although the values in the first column (procedures.BASE_COST), which represents the line item cost of a procedure, numerically match values in the second column (Daily_Town_COVID19.covid_cases_cumulative), which records the total number of COVID-19 cases, these columns are not logically related. The similarity in numbers does not imply a meaningful or relevant relationship between the costs of medical procedures and the cumulative count of COVID-19 cases.



### Final Result

After using IND with MIND algorithem and then validation then analysis the results this is the final relationship between columns across different tables:

- encounters.PAYER -> payers.Id
- patients.STATE -> COVID_19_Test_to_Treat.STATE
- patients.COUNTY -> Daily_Town_COVID19.COUNTY
- payers.Id -> encounters.PAYER
- procedures.Year-Month -> encounters.Year-Month
- COVID_19_Test_to_Treat.CITY -> patients.CITY
- COVID_19_Test_to_Treat.STATE -> patients.STATE
- COVID_19_Test_to_Treat.CITY -> Daily_Town_COVID19.Town
- Daily_Town_COVID19.COUNTY -> patients.COUNTY

In [ ]:
folder_name = 'Valid Data'

# Check if the folder exists, and if not, create it
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Save each dataframe to a CSV file in the "Valid Data" folder
encounters.to_csv(f'{folder_name}/encounters-valid.csv', index=False)
patients.to_csv(f'{folder_name}/patients-valid.csv', index=False)
payers.to_csv(f'{folder_name}/payers-valid.csv', index=False)
procedures.to_csv(f'{folder_name}/procedures-valid.csv', index=False)
COVID_19_Test_to_Treat.to_csv(f'{folder_name}/COVID_19_Test_to_Treat-valid.csv', index=False)
Daily_Town_COVID19.to_csv(f'{folder_name}/Daily_Town_COVID19-valid.csv', index=False)

# Create Relational Database

## Step 1: Define Tables and Relationships

### Tables:
1. **Encounters**
   - Columns: Id (Primary Key), PAYER (Foreign Key to Payers), PATIENT (Foreign Key to Patients), Year-Month, other encounter-specific attributes.
  
2. **Patients**
   - Columns: Id (Primary Key), STATE (Foreign Key to COVID_19_Test_to_Treat, aligning with STATE), COUNTY (Foreign Key to Daily_Town_COVID19, aligning with COUNTY), CITY, and other patient-specific attributes.
  
3. **Payers**
   - Columns: Id (Primary Key), NAME, ADDRESS, and other payer-specific attributes.

4. **Procedures**
   - Columns: Id (Primary Key), ENCOUNTER (Foreign Key to Encounters), Year-Month, and other procedure-specific attributes.

5. **COVID_19_Test_to_Treat**
   - Columns: Id (Primary Key), STATE, CITY, and other relevant COVID-19 testing information.

6. **Daily_Town_COVID19**
   - Columns: Id (Primary Key), COUNTY, Town, and other daily COVID-19-specific attributes.

## Step 2: Set Up Foreign Key Constraints

- **Encounters.PAYER** links to **Payers.Id**.
- **Patients.STATE** and **Patients.COUNTY** link respectively to **COVID_19_Test_to_Treat.STATE** and **Daily_Town_COVID19.COUNTY**.
- **Procedures.Year-Month** correlates with **Encounters.Year-Month**.
- **COVID_19_Test_to_Treat.CITY** and **COVID_19_Test_to_Treat.STATE** correlate with **Patients.CITY** and **Patients.STATE**.
- **COVID_19_Test_to_Treat.CITY** links to **Daily_Town_COVID19.Town**.

In [2]:
# Initialize Base for ORM
Base = declarative_base()

# Define the tables

class Payer(Base):
    __tablename__ = 'payers'
    Id = Column(String, primary_key=True)
    NAME = Column(String)
    ADDRESS = Column(String)
    CITY = Column(String)
    STATE_HEADQUARTERED = Column(String)
    ZIP = Column(String)
    PHONE = Column(String)


class Encounter(Base):
    __tablename__ = 'encounters'
    Id = Column(String, primary_key=True)
    START = Column(Date)
    STOP = Column(Date)
    PATIENT = Column(String, ForeignKey('patients.Id'))
    ORGANIZATION = Column(String)
    PAYER = Column(String, ForeignKey('payers.Id'))
    ENCOUNTERCLASS = Column(String)
    CODE = Column(String)
    DESCRIPTION = Column(String)
    BASE_ENCOUNTER_COST = Column(Float)
    TOTAL_CLAIM_COST = Column(Float)
    PAYER_COVERAGE = Column(Float)
    REASONCODE = Column(String)
    REASONDESCRIPTION = Column(String)
    Year_Month = Column(String)


class Patient(Base):
    __tablename__ = 'patients'
    Id = Column(String, primary_key=True)
    BIRTHDATE = Column(Date)
    DEATHDATE = Column(Date)
    PREFIX = Column(String)
    FIRST = Column(String)
    LAST = Column(String)
    SUFFIX = Column(String)
    MAIDEN = Column(String)
    MARITAL = Column(String)
    RACE = Column(String)
    ETHNICITY = Column(String)
    GENDER = Column(String)
    BIRTHPLACE = Column(String)
    ADDRESS = Column(String)
    CITY = Column(String)
    STATE = Column(String)
    COUNTY = Column(String)
    ZIP = Column(String)
    LAT = Column(Float)
    LON = Column(Float)
    Age = Column(Integer)
    Age_Group = Column(String)


class Procedure(Base):
    __tablename__ = 'procedures'
    Id = Column(String, primary_key=True)
    START = Column(Date)
    STOP = Column(Date)
    PATIENT = Column(String, ForeignKey('patients.Id'))
    ENCOUNTER = Column(String, ForeignKey('encounters.Id'))
    CODE = Column(String)
    DESCRIPTION = Column(String)
    BASE_COST = Column(Float)
    REASONCODE = Column(String)
    REASONDESCRIPTION = Column(String)
    Year_Month = Column(String)


class COVID19TestToTreat(Base):
    __tablename__ = 'COVID_19_Test_to_Treat'
    Provider_Name = Column(String, primary_key=True)
    Address1 = Column(String)
    CITY = Column(String)
    STATE = Column(String)
    Zip = Column(String)
    Last_Report_Date = Column(Date)
    Geopoint = Column(String)


class DailyTownCOVID19(Base):
    __tablename__ = 'Daily_Town_COVID19'
    Date = Column(Date, primary_key=True)
    COUNTY = Column(String)
    Town = Column(String)
    covid_cases_cumulative = Column(Integer)
    daily_cases = Column(Integer)
    Year_Month = Column(String)

# Create the database engine
engine = create_engine('sqlite:///health_data.db')

# Create all tables
Base.metadata.create_all(engine)

print("Database and tables have been created.")

Database and tables have been created.


## Load Data Into The Tables

In [3]:
inspector = inspect(engine)
print(inspector.get_table_names())


['COVID_19_Test_to_Treat', 'Daily_Town_COVID19', 'encounters', 'patients', 'payers', 'procedures']


In [4]:
file_paths = {
    'payers': 'Valid Data/payers-valid.csv',
    'encounters': 'Valid Data/encounters-valid.csv',
    'patients': 'Valid Data/patients-valid.csv',
    'procedures': 'Valid Data/procedures-valid.csv',
    'COVID_19_Test_to_Treat': 'Valid Data/COVID_19_Test_to_Treat-valid.csv',
    'Daily_Town_COVID19': 'Valid Data/Daily_Town_COVID19-valid.csv'
}

In [ ]:
# Dictionary to store data types for each file
data_types = {}

for key, file_path in file_paths.items():
    # Load each file
    df = pd.read_csv(file_path)
    # Get data types for each column
    data_types[key] = df.dtypes

# Display data types for each file
for key, dtypes in data_types.items():
    print(f"Data types for {key}:\n{dtypes}\n")

Data types for payers:
Id                     object
NAME                   object
ADDRESS                object
CITY                   object
STATE_HEADQUARTERED    object
ZIP                    object
PHONE                  object
dtype: object

Data types for encounters:
Id                      object
START                   object
STOP                    object
PATIENT                 object
ORGANIZATION            object
PAYER                   object
ENCOUNTERCLASS          object
CODE                     int64
DESCRIPTION             object
BASE_ENCOUNTER_COST    float64
TOTAL_CLAIM_COST       float64
PAYER_COVERAGE         float64
REASONCODE             float64
REASONDESCRIPTION       object
Year-Month              object
dtype: object

Data types for patients:
Id             object
BIRTHDATE      object
DEATHDATE      object
PREFIX         object
FIRST          object
LAST           object
SUFFIX         object
MAIDEN         object
MARITAL        object
RACE           object


In [ ]:
# Load data into tables
for table_name, file_path in file_paths.items():
    data = pd.read_csv(file_path)
    data.to_sql(table_name, con=engine, if_exists='replace', index=False)

print("Data has been loaded into the database.")



Data has been loaded into the database.


## Example

In [ ]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', 1000)  

In [ ]:
# Check table columns
tables = ["encounters", "patients", "payers", "procedures", "COVID_19_Test_to_Treat", "Daily_Town_COVID19"]
for table in tables:
    df = pd.read_sql(f"SELECT * FROM {table}", engine)
    print(f"Columns in {table}: {df.columns.tolist()}")

Columns in encounters: ['Id', 'START', 'STOP', 'PATIENT', 'ORGANIZATION', 'PAYER', 'ENCOUNTERCLASS', 'CODE', 'DESCRIPTION', 'BASE_ENCOUNTER_COST', 'TOTAL_CLAIM_COST', 'PAYER_COVERAGE', 'REASONCODE', 'REASONDESCRIPTION', 'Year-Month']
Columns in patients: ['Id', 'BIRTHDATE', 'DEATHDATE', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP', 'LAT', 'LON', 'Age', 'Age Group']
Columns in payers: ['Id', 'NAME', 'ADDRESS', 'CITY', 'STATE_HEADQUARTERED', 'ZIP', 'PHONE']
Columns in procedures: ['START', 'STOP', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION', 'BASE_COST', 'REASONCODE', 'REASONDESCRIPTION', 'Year-Month']
Columns in COVID_19_Test_to_Treat: ['Provider Name', 'Address1', 'CITY', 'STATE', 'Zip', 'Last Report Date', 'Geopoint']
Columns in Daily_Town_COVID19: ['Date', 'COUNTY', 'Town', 'covid_cases_cumulative', 'daily_cases', 'Year-Month']


In [ ]:
with engine.connect() as connection:
    df = pd.read_sql("SELECT * FROM payers LIMIT 5", con=connection)
    print(df)


                                     Id                    NAME             ADDRESS        CITY STATE_HEADQUARTERED      ZIP           PHONE
0  b3221cfc-24fb-339e-823d-bc4136cbc4ed           Dual Eligible  7500 Security Blvd   Baltimore                  MD  21244.0  1-877-267-2323
1  7caa7254-5050-3b5e-9eae-bd5ea30e809c                Medicare  7500 Security Blvd   Baltimore                  MD  21244.0  1-800-633-4227
2  7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a                Medicaid  7500 Security Blvd   Baltimore                  MD  21244.0  1-877-267-2323
3  d47b3510-2895-3b70-9897-342d681c769d                  Humana    500 West Main St  Louisville                  KY  40018.0  1-844-330-7799
4  6e2f1a2d-27bd-3701-8d08-dae202c58632  Blue Cross Blue Shield      Michigan Plaza     Chicago                  IL  60007.0  1-800-262-2583


In [ ]:
with engine.connect() as connection:
    df = pd.read_sql("SELECT * FROM encounters LIMIT 5", con=connection)
    print(df)

                                     Id                START                 STOP                               PATIENT                          ORGANIZATION                                 PAYER ENCOUNTERCLASS       CODE                                 DESCRIPTION  BASE_ENCOUNTER_COST  TOTAL_CLAIM_COST  PAYER_COVERAGE  REASONCODE REASONDESCRIPTION Year-Month
0  32c84703-2481-49cd-d571-3899d5820253  2011-01-02 09:26:36  2011-01-02 12:58:36  3de74169-7f67-9304-91d4-757e0f3a14d2  d78e84ec-30aa-3bba-a33a-f29a3a454662  b1c428d6-4f07-31e0-90f0-68ffa6ff8c76     ambulatory  185347001           Encounter for problem (procedure)                85.55           1018.02             0.0        -1.0              None    2011-01
1  a9183b4f-2572-72ea-54c2-b3cd038b4be7  2011-01-03 17:36:53  2011-01-03 17:51:53  fa006887-d93c-d302-8b89-f3c25f88c0e1  d78e84ec-30aa-3bba-a33a-f29a3a454662  42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a     ambulatory  390906007                         Follow-up encounter           

In [ ]:
with engine.connect() as connection:
    df = pd.read_sql("SELECT * FROM patients LIMIT 5", con=connection)
    print(df)

                                     Id   BIRTHDATE   DEATHDATE PREFIX      FIRST           LAST SUFFIX     MAIDEN MARITAL   RACE    ETHNICITY GENDER                    BIRTHPLACE                       ADDRESS    CITY          STATE   COUNTY     ZIP        LAT        LON  Age Age Group
0  5605b66b-e92d-c16c-1b83-b8bf7040d51f  1977-03-19               Mrs.  Nikita578      Erdman779         Leannon79       M  white  nonhispanic      F  Wakefield  Massachusetts  US    510 Little Station Unit 69  Quincy  Massachusetts  Norfolk  2186.0  42.290937 -70.975503   48     35-49
1  6e5ae27c-8038-7988-e2c0-25a103f01bfa  1940-02-19                Mr.    Zane918  Hodkiewicz467                         M  white  nonhispanic      M  Brookline  Massachusetts  US           747 Conn Throughway  Boston  Massachusetts  Suffolk  2135.0  42.308831 -71.063162   85       80+
2  8123d076-0886-9007-e956-d5864aa121a7  1958-06-04                Mr.   Quinn173   Marquardt819                         M  white  nonhispa

In [ ]:
with engine.connect() as connection:
    df = pd.read_sql("SELECT * FROM procedures LIMIT 5", con=connection)
    print(df)

                 START                 STOP                               PATIENT                             ENCOUNTER             CODE                                        DESCRIPTION  BASE_COST  REASONCODE REASONDESCRIPTION Year-Month
0  2011-01-12 18:09:27  2011-01-12 19:05:48  712e95dd-f313-39c7-1fc9-8463c15c105f  44057ea7-178f-c3c3-965a-78e4521fa43b        710824005  Assessment of health and social care needs (pr...        431        -1.0              None    2011-01
1  2011-01-12 19:05:48  2011-01-12 19:30:35  712e95dd-f313-39c7-1fc9-8463c15c105f  44057ea7-178f-c3c3-965a-78e4521fa43b        762993000      Assessment using Morse Fall Scale (procedure)        431        -1.0              None    2011-01
2  2011-01-12 19:30:35  2011-01-12 20:09:20  712e95dd-f313-39c7-1fc9-8463c15c105f  44057ea7-178f-c3c3-965a-78e4521fa43b        866148006           Screening for domestic abuse (procedure)        431        -1.0              None    2011-01
3  2011-01-12 20:09:20  2011-01-12 20:22

In [ ]:
with engine.connect() as connection:
    df = pd.read_sql("SELECT * FROM covid_19_test_to_treat LIMIT 5", con=connection)
    print(df)

                                  Provider Name             Address1       CITY          STATE       Zip        Last Report Date                      Geopoint
0                              WALGREENS #19487     330 RIVER STREET  Cambridge  Massachusetts  21394618  01/30/2024 12:00:00 AM     POINT (-71.1139 42.36167)
1                           WATCH CITY PHARMACY          923 MAIN ST    Waltham  Massachusetts      2451  01/30/2024 12:00:00 AM   POINT (-71.248206 42.37648)
2          NORTH END WATERFRONT HEALTH PHARMACY       332 HANOVER ST     Boston  Massachusetts  21131901  01/30/2024 12:00:00 AM  POINT (-71.053742 42.364825)
3  CAMBRIDGE HEALTH ALLIANCE SPECIALTY PHARMACY  350 MAIN ST STE 640     Malden  Massachusetts      2148  01/30/2024 12:00:00 AM  POINT (-71.068263 42.425658)
4                                  GARY DRUG CO        59 CHARLES ST     Boston  Massachusetts  21144605  01/30/2024 12:00:00 AM  POINT (-71.070057 42.357655)


In [ ]:
with engine.connect() as connection:
    df = pd.read_sql("SELECT * FROM daily_town_covid19 LIMIT 5", con=connection)
    print(df)

         Date COUNTY      Town  covid_cases_cumulative  daily_cases Year-Month
0  2020-04-01  Essex  Amesbury                       6            6    2020-04
1  2020-04-01  Essex   Andover                      13           13    2020-04
2  2020-04-01  Essex   Beverly                      37           37    2020-04
3  2020-04-01  Essex   Boxford                       4            4    2020-04
4  2020-04-01  Essex   Danvers                      47           47    2020-04
